# Scraping Listing Prices
Before being able to do any advanced analysis, we need data!
[Home](https://www.home.dk) is one of the largest Real Estate companies in Denmark with listings from all over the country. 

Using the requests library we can query their database and save the result to a pandas dataframe.

Let's dive in!

## Libraries

In [6]:
# Importing libraries
import pandas as pd
import requests
import bs4
import time

Browsing Home, the largest real estate company in Denmark and playing arround with the developer tools, I managed to find HTTP call that seem to return the data of the listings.

In [7]:
# Using the home, the biggest real estate company in Denmark
url = ('https://home.dk/umbraco/backoffice/home-api/SEARCH?' 
       'CurrentPageNumber=0&SearchResultsPerPage=10&q=2200&'
       'Energimaerker=null&SearchType=0&_=1571481546474')
response = requests.get(url)
# Saving response to a dictionary
featuresDict = response.json()

In [8]:
# Checking our the data
featuresDict

{'redirectUrl': None,
 'inputModel': {'SortType': None,
  'SortOrder': None,
  'CurrentPageNumber': 0,
  'SearchResultsPerPage': 10,
  'q': '2200',
  'EjendomstypeV1': None,
  'EjendomstypeRH': None,
  'EjendomstypeEL': None,
  'EjendomstypeVL': None,
  'EjendomstypeAA': None,
  'EjendomstypePL': None,
  'EjendomstypeFH': None,
  'EjendomstypeLO': None,
  'EjendomstypeHG': None,
  'EjendomstypeFG': None,
  'EjendomstypeNL': None,
  'Forretningnr': None,
  'ProjectNodeId': None,
  'OnlyBrokerHome': None,
  'IsFirstClass': None,
  'PriceMin': None,
  'PriceMax': None,
  'EjerudgiftPrMdrMin': None,
  'EjerudgiftPrMdrMax': None,
  'BoligydelsePrMdrMin': None,
  'BoligydelsePrMdrMax': None,
  'BoligstoerrelseMin': None,
  'BoligstoerrelseMax': None,
  'GrundstoerrelseMin': None,
  'GrundstoerrelseMax': None,
  'VaerelserMin': None,
  'VaerelserMax': None,
  'Energimaerker': ['null'],
  'ByggaarMin': None,
  'ByggaarMax': None,
  'EtageMin': None,
  'EtageMax': None,
  'PlanMin': None,
  'Pl

What we want to extract seem to be withing the searchResult key:

In [9]:
featuresDict['searchResults']

[{'sagsnummer': '1280000094',
  'lng': 12.5443935823075,
  'lat': 55.6874081312121,
  'fokusbolig': False,
  'showNewPrice': False,
  'isNew': True,
  'adresse': 'Hiort Lorenzens Gade 2, 1. tv.',
  'postal': 2200,
  'city': 'København N',
  'price': '5.795.000 ',
  'ejendomstypePrimaerNicename': 'Ejerlejlighed',
  'pictures': [{'PicId': 3012507,
    'CaseId': 10399600,
    'CaseNumber': '1280000094',
    'MediaType': 'b',
    'MaxWidth': 3000,
    'MaxHeight': 2000,
    'URL': 'https://home.mindworking.eu/resources/shops/128/cases/1280000094/casemedia/images/f5c395785295f1e50f54c45433d490fd/customsize.jpg?deviceId=jd83hsdf3',
    'Position': 0,
    'Description': 'Ejendommen',
    'GUID': 'f5c39578-5295-f1e5-0f54-c45433d490fd',
    'refGUID': '00000000-0000-0000-0000-000000000000',
    'IsVertical': False,
    'IsHorizontal': True},
   {'PicId': 3012027,
    'CaseId': 10399600,
    'CaseNumber': '1280000094',
    'MediaType': 'b',
    'MaxWidth': 3000,
    'MaxHeight': 2000,
    'URL':

Great! This is the data we're interested in. However the pictures key contain a list of information, we don't need which would ruin the granularity should we convert it to a pandas Dataframe so let's drop it.

In [10]:
# dropping the pictures key from the list of dictionaries
features = featuresDict['searchResults']
for f in features:
    del f['pictures']
features

[{'sagsnummer': '1280000094',
  'lng': 12.5443935823075,
  'lat': 55.6874081312121,
  'fokusbolig': False,
  'showNewPrice': False,
  'isNew': True,
  'adresse': 'Hiort Lorenzens Gade 2, 1. tv.',
  'postal': 2200,
  'city': 'København N',
  'price': '5.795.000 ',
  'ejendomstypePrimaerNicename': 'Ejerlejlighed',
  'floorPlan': {'PicId': 3012769,
   'CaseId': 10399600,
   'CaseNumber': '1280000094',
   'MediaType': 'p',
   'MaxWidth': 3000,
   'MaxHeight': 2000,
   'URL': 'https://home.mindworking.eu/resources/shops/128/cases/1280000094/casemedia/images/3bd89cc82d743456c13bad5f5dbc36b5/customsize.jpg?deviceId=jd83hsdf3',
   'Position': 0,
   'Description': 'Plantegning',
   'GUID': '3bd89cc8-2d74-3456-c13b-ad5f5dbc36b5',
   'refGUID': '00000000-0000-0000-0000-000000000000',
   'IsVertical': False,
   'IsHorizontal': True},
  'boligOrGrundAreal': 147,
  'andenmaegler': False,
  'boligurl': 'https://home.dk/boligkatalog/koebenhavn/2200/ejerlejligheder/hiort_lorenzens_gade_2_1_tv_128000009

The data seem ready to be loaded to a pandas dataframe.

In [11]:
df = pd.DataFrame(features)
df.head()

,aabenthusNicename,aabenthusShowRegistration,adresse,andenmaegler,billedeUrl,boligKanLejes,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,...,lejePerMaaned,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer,showNewPrice,solgtBolig
0,10.11 kl. 11.30-12.00,False,"Hiort Lorenzens Gade 2, 1. tv.",False,https://home.mindworking.eu/resources/shops/12...,0,147,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.544394,2019-11-10T12:00,2019-11-10T11:30,"Oplagt til børnefamilien, med mulighed for 2 g...",2200,5.795.000,1280000094,False,False
1,10.11 kl. 12.30-12.50,False,"Haraldsgade 4, st.. th.",False,https://home.mindworking.eu/resources/shops/10...,0,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,...,,12.548805,2019-11-10T12:50,2019-11-10T12:30,,2200,3.146.632,1050000163,False,False
2,10.11 kl. 15.00-15.20,False,"Farumgade 1, 3. tv.",False,https://home.mindworking.eu/resources/shops/10...,0,52,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.542814,2019-11-10T15:20,2019-11-10T15:00,Super lav ejerudgift!,2200,2.299.000,1050000165,False,False
3,10.11 kl. 13.30-13.50,False,"Møllegade 22, st..",False,https://home.mindworking.eu/resources/shops/10...,0,116,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.558514,2019-11-10T13:50,2019-11-10T13:30,,2200,4.550.000,1050000152,False,False
4,10.11 kl. 12.00-12.20,False,"Bjelkes Allé 6B, st..",False,https://home.mindworking.eu/resources/shops/10...,0,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,...,,12.545717,2019-11-10T12:20,2019-11-10T12:00,,2200,2.095.000,1050000139,False,False


Let's remove columns that are not of interest.

In [14]:
df.drop(inplace = True, columns=[
    'billedeUrl','lejePerMaaned','showNewPrice',
    'aabenthusNicename','floorPlan','erSolgtOgLejebolig',
    'boligKanLejes','aabenthusShowRegistration', 
    'solgtBolig','isLejebolig','fokusbolig'
])

In [15]:
df.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isFirstClass,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"Hiort Lorenzens Gade 2, 1. tv.",False,147,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.687408,12.544394,2019-11-10T12:00,2019-11-10T11:30,"Oplagt til børnefamilien, med mulighed for 2 g...",2200,5.795.000,1280000094
1,"Haraldsgade 4, st.. th.",False,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,False,True,55.703313,12.548805,2019-11-10T12:50,2019-11-10T12:30,,2200,3.146.632,1050000163
2,"Farumgade 1, 3. tv.",False,52,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.698103,12.542814,2019-11-10T15:20,2019-11-10T15:00,Super lav ejerudgift!,2200,2.299.000,1050000165
3,"Møllegade 22, st..",False,116,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.692386,12.558514,2019-11-10T13:50,2019-11-10T13:30,,2200,4.550.000,1050000152
4,"Bjelkes Allé 6B, st..",False,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.692485,12.545717,2019-11-10T12:20,2019-11-10T12:00,,2200,2.095.000,1050000139


The 'boligurl' is the URL to the site of each piece of real estate for sale, so let's use that to get more features!

In [19]:
response = requests.get(df['boligurl'][0])
html = response.text

In [20]:
soup = bs4.BeautifulSoup(html, "html.parser")
soup


<!DOCTYPE html>

<html class="no-js" lang="da" ng-app="home" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#">
<head>
<script async="" data-culture="DA" id="CookieConsent" src="https://policy.cookieinformation.com/uc.js"></script>
<script>(function(H){H.className=H.className.replace(/\bno-js\b/,'js')})(document.documentElement)</script>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=2" id="viewport" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>Ejerlejlighed - 2200 København N - Hiort Lorenzens Gade 2, 1. tv.</title>
<meta content="Ejerlejlighed - 2200 København N - Hiort Lorenzens Gade 2, 1. tv." name="title"/>
<meta content="" name="keywords">
<meta content='Ejerlejlighed til salg, København N - &lt;div&gt;&lt;span style="font-size: 10.4667px;"&gt;De herskabelige former møder allerede én i opgangen, som er helt nyrenoveret m

The stuff we want is in the info-property and info-value class.

In [23]:
additionalFeatures = soup.find_all('span', {"class": ["info-property","info-value"]})

They come in pairs and we need them divivded into key-value pairs.

In [25]:
# Loop through each span in the list
count = 0
keys = []
values = []
for feat in additionalFeatures:
    if count % 2: # Odd number is a value
        values.append(feat.text.strip())
    else: # Even number is a key
        keys.append(feat.text.strip())
    count +=1 
dictionary = dict(zip(keys, values))
dictionary

{'Kontantpris': '5.795.000  kr.',
 'Ejerudgift pr. md.': '4.697  kr.',
 'Kvm. pris ?': '39.421  kr.',
 'Udbetaling': '290.000  kr.',
 'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift': '22.369  / 19.585  kr.',
 'Prisudvikling': '0%',
 'Boligareal': '147  m2',
 'Antal toiletter': '2',
 'Antal rum': '5',
 'Byggeår': '1905',
 'Antal plan': '1',
 'Etage': '1',
 'Energimærke': 'D',
 'Sagsnr.': '1280000094',
 'Afstand til off. transport': '220  m',
 'Afstand til skole': '3.600  m',
 'Afstand til indkøb': '650  m',
 'Tilbehør': 'Bosch  emhætte  - Smeg  komfur  - 2001Bosch  køle/fryseskab  - Miele  opvaskemaskine  - Miele  vaskemaskine  - Miele  tørretumbler  - Alaska  mikroovn  -',
 'Ejendomsværdi i kr.': '2.650.000',
 'Heraf grundværdi i kr.': '152.300',
 'Vurderingsår': '2018'}

This should be repeated for each line in the dataframe and to be appended as columns. Let's create a function for this.

In [26]:
def get_additional_features(df):
    """
    Function for getting additional features from each of the listings. 
    Input is a pandas dataframe.
    """
    additionalFeaturesList = []
    # Counter for showing progress
    counter = 0
    loops = df.shape[0]
    # Loop through all rows
    for url in df['boligurl']:
        try:
            response = requests.get(url)
            html = response.text
            soup = bs4.BeautifulSoup(html, "html.parser")
            additionalFeatures = soup.find_all('span', {"class": ["info-property","info-value"]})

            # Loop through each span in the list
            count = 0
            keys = []
            values = []
            for feat in additionalFeatures:
                if count % 2: # Odd number is a value
                    values.append(feat.text.strip())
                else: # Even number is a key
                    keys.append(feat.text.strip())
                count +=1 
        except:
            keys.append('Connection timed out')
            values.append('True')
            
        additionalFeaturesList.append(dict(zip(keys, values)))
        time.sleep(2)
        counter += 1
        print('Progress {}'.format((float(counter)/float(loops))*100.))
    df2 = df.join(pd.DataFrame(additionalFeaturesList))
    return df2

In [27]:
df2 = get_additional_features(df)
df2.head()

Progress 10.0
Progress 20.0
Progress 30.0
Progress 40.0
Progress 50.0
Progress 60.0
Progress 70.0
Progress 80.0
Progress 90.0
Progress 100.0


,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isFirstClass,isNew,lat,lng,...,Sagsnr.,Teknisk pris ?,Tilbehør,Udbetaling,Udlejning,Udlejning tilladt,Vaskeri,Vinduer,Vurderingsår,Ydermur
0,"Hiort Lorenzens Gade 2, 1. tv.",False,147,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.687408,12.544394,...,1280000094,NaN,Bosch emhætte - Smeg komfur - 2001Bosch k...,290.000 kr.,NaN,NaN,NaN,NaN,2018,NaN
1,"Haraldsgade 4, st.. th.",False,105,https://home.dk/boligkatalog/koebenhavn/2200/a...,København N,Andelsbolig,False,True,55.703313,12.548805,...,1050000163,3.211.831 kr.,NaN,NaN,NaN,NaN,Nej,Brædder,2018,Mursten
2,"Farumgade 1, 3. tv.",False,52,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.698103,12.542814,...,1050000165,NaN,Whirlpool køle-/fryseskabSMEG komfurUPO vaskem...,115.000 kr.,Tilladt,NaN,Vaskemaskine i lejligheden,Termo,2018,Mursten
3,"Møllegade 22, st..",False,116,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.692386,12.558514,...,1050000152,NaN,Siemens keramisk bordkomfurSiemens indbygnings...,230.000 kr.,NaN,"Tilladt, lejekontrakt skal sendes til bestyrel...",Nej,NaN,2018,Mursten
4,"Bjelkes Allé 6B, st..",False,54,https://home.dk/boligkatalog/koebenhavn/2200/e...,København N,Ejerlejlighed,False,True,55.692485,12.545717,...,1050000139,NaN,Gorenje komfurAEG køle/fryseskabBosch emhætteh...,105.000 kr.,NaN,Tilladt,Ja,NaN,2018,Mursten


In [28]:
df2.columns

Index(['adresse', 'andenmaegler', 'boligOrGrundAreal', 'boligurl', 'city',
       'ejendomstypePrimaerNicename', 'isFirstClass', 'isNew', 'lat', 'lng',
       'openHouseEndDate', 'openHouseStartDate', 'overskrift2', 'postal',
       'price', 'sagsnummer', 'Afstand til indkøb',
       'Afstand til off. transport', 'Afstand til skole', 'Altan',
       'Antal plan', 'Antal rum', 'Antal toiletter', 'Antenne', 'Boligareal',
       'Boligydelse pr. måned',
       'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift',
       'Byggeår', 'Ejendomsværdi i kr.', 'Ejerudgift pr. md.', 'El',
       'Energimærke', 'Etage', 'Fibernet', 'Forurening', 'Grundareal', 'Gulve',
       'Heraf grundværdi i kr.', 'Husdyr', 'Husdyr tilladt', 'Kontantpris',
       'Kvm. pris ?', 'Købspris', 'Overtagelse', 'Prisudvikling', 'Pulterrum',
       'Sagsnr.', 'Teknisk pris ?', 'Tilbehør', 'Udbetaling', 'Udlejning',
       'Udlejning tilladt', 'Vaskeri', 'Vinduer', 'Vurderingsår', '

Alright, we can now do this entire process for multiple zip codes and more than 10 returns.

Note: Through trial and error I found the maximum number of returns to be 200 and in order to get all the data, we can use the URL to add search criteria to split our results into smaller bins.

In [141]:
# Zip codes in Denmark
zipCode = [1301,2000,2100,2200,2300,2400,2450,2500,2600,2605,2610,2625,2630,
           2635,2640,2650,2660,2665,2670,2670,2680,2690,2700,2720,2730,2740,
           2750,2760,2765,2770,2791,2800,2820,2830,2840,2850,2860,2880,2900,
           2920,2930,2942,2950,2960,2970,2980,2990,3000,3050,3060,3070,3080,
           3100,3120,3140,3150,3200,3210,3220,3230,3250,3300,3310,3320,3330,
           3360,3370,3390,3400,3460,3480,3490,3500,3520,3540,3550,3600,3630,
           3650,3660,3670,3700,3720,3730,3740,3751,3760,3770,3782,3790,4000,
           4040,4050,4060,4070,4100,4130,4140,4160,4171,4173,4174,4180,4190,
           4200,4220,4230,4241,4242,4243,4250,4261,4262,4270,4281,4291,4293,
           4295,4296,4300,4320,4330,4340,4350,4360,4370,4390,4400,4420,4440,
           4450,4460,4470,4480,4490,4500,4520,4532,4534,4540,4550,4560,4571,
           4572,4573,4581,4583,4591,4592,4593,4600,4621,4622,4623,4632,4640,
           4652,4653,4654,4660,4671,4672,4673,4681,4682,4683,4684,4690,4700,
           4720,4733,4735,4736,4750,4760,4771,4772,4773,4780,4791,4792,4793,
           4800,4840,4850,4862,4863,4871,4872,4873,4874,4880,4891,4892,4894,
           4895,4900,4912,4913,4920,4930,4941,4943,4944,4951,4952,4953,4960,
           4970,4983,4990,5000,5200,5210,5220,5230,5240,5250,5260,5270,5290,
           5300,5330,5350,5370,5380,5390,5400,5450,5462,5463,5464,5466,5471,
           5474,5485,5491,5492,5500,5540,5550,5560,5580,5591,5592,5600,5610,
           5620,5631,5642,5672,5683,5690,5700,5750,5762,5771,5772,5792,5800,
           5853,5854,5856,5863,5871,5874,5881,5882,5883,5884,5892,5900,5932,
           5935,5953,5960,5970,5985,6000,6040,6051,6052,6064,6070,6091,6092,
           6093,6094,6100,6200,6230,6240,6261,6270,6280,6300,6310,6320,6330,
           6340,6360,6372,6392,6400,6430,6440,6470,6500,6510,6520,6535,6541,
           6560,6580,6600,6621,6622,6623,6630,6640,6650,6660,6670,6682,6683,
           6690,6700,6701,6705,6710,6715,6720,6731,6740,6752,6760,6771,6780,
           6792,6800,6818,6823,6830,6840,6851,6852,6853,6854,6855,6857,6862,
           6870,6880,6893,6900,6920,6933,6940,6950,6960,6971,6973,6980,6990,
           7000,7080,7100,7120,7130,7140,7150,7160,7171,7173,7182,7183,7184,
           7190,7200,7250,7260,7270,7280,7300,7321,7323,7330,7361,7362,7400,
           7430,7441,7442,7451,7470,7480,7490,7500,7540,7550,7560,7570,7600,
           7620,7650,7660,7673,7680,7700,7730,7741,7742,7752,7755,7760,7770,
           7790,7800,7830,7840,7850,7860,7870,7884,7900,7950,7960,7970,7980,
           7990,8000,8200,8210,8220,8230,8240,8250,8260,8270,8300,8305,8310,
           8320,8330,8340,8350,8355,8361,8362,8370,8380,8381,8382,8400,8410,
           8420,8444,8450,8462,8464,8471,8472,8500,8520,8530,8541,8543,8544,
           8550,8560,8570,8581,8585,8586,8592,8600,8620,8632,8641,8643,8653,
           8654,8660,8670,8680,8700,8721,8722,8723,8732,8740,8751,8752,8762,
           8763,8765,8766,8781,8783,8800,8830,8831,8832,8840,8850,8860,8870,
           8881,8882,8883,8900,8950,8961,8963,8970,8981,8983,8990,9000,9200,
           9210,9220,9230,9240,9260,9270,9280,9293,9300,9310,9320,9330,9340,
           9352,9362,9370,9380,9381,9382,9400,9430,9440,9460,9480,9490,9492,
           9493,9500,9510,9520,9530,9541,9550,9560,9574,9575,9600,9610,9620,
           9631,9632,9640,9670,9681,9690,9700,9740,9750,9760,9800,9830,9850,
           9870,9881,9900,9940,9970,9981,9982,9990]

In [142]:
featureList = []
# Loop through zip codes
for code in zipCode:
    # If the zipcode is in one of the larger cities, 
    # split the search into chunks based on size of home
    if code in [1301, 2000, 2100, 2200, 2300, 2400, 2450, 2500,
                5000, 5200, 5210, 5220, 5230, 5240, 5250, 5260,
                5270, 8000, 8200, 8210, 8220, 8230, 8240, 9000,
                9200, 9210, 9220
               ]:
        # Setting size interval to bin responses into smaller chunks
        min_size = 11
        max_size = 20
        # Loop through sizes
        for i in range(28):
            url = ('https://home.dk/umbraco/backoffice/home-api/SEARCH?'
                   'CurrentPageNumber=0&SearchResultsPerPage=200&BoligstoerrelseMin=' 
                   + str(min_size) 
                   + '&BoligstoerrelseMax=' 
                   + str(max_size) 
                   + '&q=' 
                   + str(code) 
                   + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474')

            response = requests.get(url)
            # Saving response to a dictionary
            featuresDict = response.json()
            # dropping the pictures key from the list of dictionaries
            features = featuresDict['searchResults']
            for f in features:
                del f['pictures']
            featureList.extend(features)
            # Pausing to not be a dick towards the server
            time.sleep(1)
            
            # Count up sizes for next bin
            min_size += 10
            max_size += 10
        
        # Run one additional time with out the maximum boundry
        url = ('https://home.dk/umbraco/backoffice/home-api/SEARCH?'
               'CurrentPageNumber=0&SearchResultsPerPage=200&BoligstoerrelseMin=' 
               + str(min_size) 
               + '&q=' 
               + str(code) 
               + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474')

        response = requests.get(url)
        # Saving response to a dictionary
        featuresDict = response.json()
        # dropping the pictures key from the list of dictionaries
        features = featuresDict['searchResults']
        for f in features:
            del f['pictures']
        featureList.extend(features)
    # If the zipcode not in a larger city
    else:
        url = ('https://home.dk/umbraco/backoffice/home-api/SEARCH?'
               'CurrentPageNumber=0&SearchResultsPerPage=200&q='
               + str(code) 
               + '&Energimaerker=null&SortOrder=asc&SearchType=0&_=1571481546474')

        response = requests.get(url)
        # Saving response to a dictionary
        featuresDict = response.json()
        # dropping the pictures key from the list of dictionaries
        features = featuresDict['searchResults']
        for f in features:
            del f['pictures']
        featureList.extend(features)
# Print number of listings
len(featureList)

51207

In [143]:
df_new = pd.DataFrame(featureList)
df_new.drop(inplace = True, columns=[
    'billedeUrl','lejePerMaaned','showNewPrice',
    'aabenthusNicename','floorPlan','erSolgtOgLejebolig',
    'boligKanLejes','aabenthusShowRegistration', 
    'solgtBolig','isLejebolig','fokusbolig'
])
df_new.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,,,,2000,2.145.000,1740000062
2,"Holger Danskes Vej 12, 3 tv",True,46.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.686592,12.538423,,,None,2000,2.150.000,20002433_10007
3,"Ane Katrines Vej 16, 2 4",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,,,None,2000,2.099.000,303562_100910_10009
4,"Ane Katrines Vej 16, St. 1",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,,,None,2000,2.295.000,100088_103402_10009


In [144]:
# Save data in order to not repeat previous steps
df_new.to_csv('baseData.csv')

In [3]:
df_new = pd.read_csv('baseData.csv',index_col = 0 )
df_new.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,NaN,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,NaN,NaN,NaN,2000,2.145.000,1740000062
2,"Holger Danskes Vej 12, 3 tv",True,46.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.686592,12.538423,NaN,NaN,NaN,2000,2.150.000,20002433_10007
3,"Ane Katrines Vej 16, 2 4",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,NaN,NaN,NaN,2000,2.099.000,303562_100910_10009
4,"Ane Katrines Vej 16, St. 1",True,45.0,https://www.boligsiden.dk/viderestillingekster...,Frederiksberg,Ejerlejlighed,False,55.691713,12.535145,NaN,NaN,NaN,2000,2.295.000,100088_103402_10009


Inspecting the data it's clear that we have some appartments from other real estate agencies, but how many? ('andenmaegler' tranlates to other real estate agency)

In [17]:
df_new.query('andenmaegler == True')['adresse'].count()

41777

Turns out it's actually the majority. It's not very likely the previously created function for getting additional features would work for the remaining URLs.

For now we can get the additional features for the listings from Home.

In [18]:
df_new.query('andenmaegler == False')['adresse'].count()

9430

In [19]:
df_home = df_new.query('andenmaegler == False')
df_home.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,openHouseStartDate,overskrift2,postal,price,sagsnummer
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,2019-10-27T11:00,NaN,2000,1.350.000,1300000111
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,NaN,NaN,NaN,2000,2.145.000,1740000062
11,"Lyøvej 5, st.. tv.",False,60.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,NaN,NaN,HØJ STUELEJLIGHED MED STOR ALTAN - LAV EJERUDG...,2000,2.875.000,1300000128
12,"H. Schneekloths Vej 13, 5. th.",False,56.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,NaN,NaN,Lys og indflytningsklar stand – perfekt delele...,2000,2.750.000,130D01015
30,"Howitzvej 61, 3. th.",False,67.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2019-11-03T15:20,2019-11-03T15:00,Nyere bad & køkken - i kort afstand til Freder...,2000,3.195.000,1300000176


In [26]:
df_home_add = get_additional_features(df_home)

Progress 0.010604453870625662
Progress 0.021208907741251323
Progress 0.03181336161187699
Progress 0.04241781548250265
Progress 0.05302226935312832
Progress 0.06362672322375398
Progress 0.07423117709437964
Progress 0.0848356309650053
Progress 0.09544008483563096
Progress 0.10604453870625664
Progress 0.11664899257688228
Progress 0.12725344644750797
Progress 0.1378579003181336
Progress 0.14846235418875928
Progress 0.15906680805938495
Progress 0.1696712619300106
Progress 0.18027571580063625
Progress 0.19088016967126192
Progress 0.20148462354188762
Progress 0.2120890774125133
Progress 0.2226935312831389
Progress 0.23329798515376457
Progress 0.24390243902439024
Progress 0.25450689289501593
Progress 0.2651113467656416
Progress 0.2757158006362672
Progress 0.28632025450689286
Progress 0.29692470837751855
Progress 0.3075291622481442
Progress 0.3181336161187699
Progress 0.3287380699893956
Progress 0.3393425238600212
Progress 0.34994697773064687
Progress 0.3605514316012725
Progress 0.3711558854718

Progress 3.138918345705196
Progress 3.149522799575822
Progress 3.160127253446447
Progress 3.1707317073170733
Progress 3.1813361611876987
Progress 3.1919406150583245
Progress 3.2025450689289503
Progress 3.2131495227995757
Progress 3.223753976670202
Progress 3.234358430540827
Progress 3.244962884411453
Progress 3.255567338282078
Progress 3.2661717921527043
Progress 3.2767762460233296
Progress 3.2873806998939554
Progress 3.2979851537645812
Progress 3.3085896076352066
Progress 3.319194061505833
Progress 3.3297985153764578
Progress 3.340402969247084
Progress 3.3510074231177094
Progress 3.361611876988335
Progress 3.3722163308589606
Progress 3.3828207847295864
Progress 3.3934252386002126
Progress 3.4040296924708375
Progress 3.414634146341464
Progress 3.425238600212089
Progress 3.435843054082715
Progress 3.4464475079533403
Progress 3.457051961823966
Progress 3.4676564156945915
Progress 3.4782608695652173
Progress 3.4888653234358427
Progress 3.4994697773064685
Progress 3.5100742311770947
Progre

Progress 6.341463414634147
Progress 6.352067868504772
Progress 6.362672322375397
Progress 6.373276776246023
Progress 6.383881230116649
Progress 6.394485683987275
Progress 6.405090137857901
Progress 6.415694591728525
Progress 6.426299045599151
Progress 6.436903499469778
Progress 6.447507953340404
Progress 6.458112407211028
Progress 6.468716861081654
Progress 6.47932131495228
Progress 6.489925768822906
Progress 6.5005302226935315
Progress 6.511134676564156
Progress 6.521739130434782
Progress 6.5323435843054085
Progress 6.542948038176035
Progress 6.553552492046659
Progress 6.564156945917285
Progress 6.574761399787911
Progress 6.585365853658537
Progress 6.5959703075291625
Progress 6.606574761399788
Progress 6.617179215270413
Progress 6.6277836691410394
Progress 6.638388123011666
Progress 6.64899257688229
Progress 6.6595970307529155
Progress 6.670201484623542
Progress 6.680805938494168
Progress 6.691410392364794
Progress 6.702014846235419
Progress 6.712619300106044
Progress 6.72322375397667

Progress 9.575821845174975
Progress 9.586426299045598
Progress 9.597030752916226
Progress 9.607635206786851
Progress 9.618239660657476
Progress 9.628844114528102
Progress 9.639448568398727
Progress 9.650053022269352
Progress 9.66065747613998
Progress 9.671261930010605
Progress 9.68186638388123
Progress 9.692470837751856
Progress 9.703075291622481
Progress 9.713679745493108
Progress 9.724284199363732
Progress 9.734888653234359
Progress 9.745493107104984
Progress 9.75609756097561
Progress 9.766702014846237
Progress 9.77730646871686
Progress 9.787910922587487
Progress 9.798515376458113
Progress 9.809119830328738
Progress 9.819724284199363
Progress 9.830328738069989
Progress 9.840933191940614
Progress 9.851537645811241
Progress 9.862142099681867
Progress 9.872746553552492
Progress 9.883351007423117
Progress 9.893955461293743
Progress 9.90455991516437
Progress 9.915164369034994
Progress 9.92576882290562
Progress 9.936373276776246
Progress 9.946977730646871
Progress 9.957582184517497
Progres

Progress 12.725344644750795
Progress 12.735949098621422
Progress 12.746553552492045
Progress 12.757158006362673
Progress 12.767762460233298
Progress 12.778366914103923
Progress 12.78897136797455
Progress 12.799575821845174
Progress 12.810180275715801
Progress 12.820784729586427
Progress 12.83138918345705
Progress 12.841993637327679
Progress 12.852598091198303
Progress 12.86320254506893
Progress 12.873806998939555
Progress 12.884411452810179
Progress 12.895015906680808
Progress 12.905620360551431
Progress 12.916224814422057
Progress 12.926829268292684
Progress 12.937433722163307
Progress 12.948038176033934
Progress 12.95864262990456
Progress 12.969247083775185
Progress 12.979851537645812
Progress 12.990455991516436
Progress 13.001060445387063
Progress 13.011664899257688
Progress 13.022269353128312
Progress 13.032873806998941
Progress 13.043478260869565
Progress 13.054082714740192
Progress 13.064687168610817
Progress 13.07529162248144
Progress 13.08589607635207
Progress 13.09650053022269

...

Progress 86.28844114528101
Progress 86.29904559915164
Progress 86.30965005302227
Progress 86.3202545068929
Progress 86.33085896076352
Progress 86.34146341463415
Progress 86.35206786850476
Progress 86.3626723223754
Progress 86.37327677624602
Progress 86.38388123011664
Progress 86.39448568398727
Progress 86.4050901378579
Progress 86.41569459172854
Progress 86.42629904559915
Progress 86.43690349946978
Progress 86.4475079533404
Progress 86.45811240721103
Progress 86.46871686108165
Progress 86.47932131495229
Progress 86.4899257688229
Progress 86.50053022269353
Progress 86.51113467656415
Progress 86.52173913043478
Progress 86.53234358430541
Progress 86.54294803817604
Progress 86.55355249204666
Progress 86.56415694591728
Progress 86.57476139978792
Progress 86.58536585365853
Progress 86.59597030752916
Progress 86.60657476139978
Progress 86.61717921527041
Progress 86.62778366914104
Progress 86.63838812301167
Progress 86.64899257688229
Progress 86.65959703075292
Progress 86.67020148462355
Progre

Progress 89.53340402969248
Progress 89.5440084835631
Progress 89.55461293743372
Progress 89.56521739130436
Progress 89.57582184517497
Progress 89.5864262990456
Progress 89.59703075291623
Progress 89.60763520678685
Progress 89.61823966065748
Progress 89.6288441145281
Progress 89.63944856839873
Progress 89.65005302226935
Progress 89.66065747613999
Progress 89.6712619300106
Progress 89.68186638388123
Progress 89.69247083775186
Progress 89.70307529162248
Progress 89.7136797454931
Progress 89.72428419936374
Progress 89.73488865323435
Progress 89.74549310710498
Progress 89.75609756097562
Progress 89.76670201484623
Progress 89.77730646871687
Progress 89.78791092258749
Progress 89.79851537645811
Progress 89.80911983032874
Progress 89.81972428419937
Progress 89.83032873806998
Progress 89.84093319194062
Progress 89.85153764581125
Progress 89.86214209968186
Progress 89.8727465535525
Progress 89.88335100742312
Progress 89.89395546129374
Progress 89.90455991516437
Progress 89.915164369035
Progress 

Progress 92.77836691410393
Progress 92.78897136797455
Progress 92.79957582184517
Progress 92.81018027571581
Progress 92.82078472958642
Progress 92.83138918345705
Progress 92.84199363732768
Progress 92.8525980911983
Progress 92.86320254506893
Progress 92.87380699893956
Progress 92.88441145281018
Progress 92.89501590668081
Progress 92.90562036055144
Progress 92.91622481442205
Progress 92.92682926829269
Progress 92.9374337221633
Progress 92.94803817603393
Progress 92.95864262990456
Progress 92.96924708377519
Progress 92.97985153764581
Progress 92.99045599151644
Progress 93.00106044538707
Progress 93.01166489925768
Progress 93.02226935312832
Progress 93.03287380699894
Progress 93.04347826086956
Progress 93.05408271474019
Progress 93.06468716861082
Progress 93.07529162248144
Progress 93.08589607635207
Progress 93.0965005302227
Progress 93.10710498409331
Progress 93.11770943796395
Progress 93.12831389183457
Progress 93.1389183457052
Progress 93.14952279957582
Progress 93.16012725344645
Progr

Progress 96.02332979851538
Progress 96.033934252386
Progress 96.04453870625663
Progress 96.05514316012726
Progress 96.06574761399787
Progress 96.07635206786851
Progress 96.08695652173913
Progress 96.09756097560975
Progress 96.10816542948038
Progress 96.118769883351
Progress 96.12937433722163
Progress 96.13997879109226
Progress 96.15058324496289
Progress 96.1611876988335
Progress 96.17179215270414
Progress 96.18239660657476
Progress 96.19300106044538
Progress 96.20360551431601
Progress 96.21420996818664
Progress 96.22481442205726
Progress 96.23541887592789
Progress 96.24602332979852
Progress 96.25662778366915
Progress 96.26723223753977
Progress 96.27783669141039
Progress 96.28844114528103
Progress 96.29904559915164
Progress 96.30965005302227
Progress 96.3202545068929
Progress 96.33085896076352
Progress 96.34146341463415
Progress 96.35206786850478
Progress 96.3626723223754
Progress 96.37327677624602
Progress 96.38388123011666
Progress 96.39448568398727
Progress 96.4050901378579
Progress 

Progress 99.26829268292683
Progress 99.27889713679745
Progress 99.28950159066808
Progress 99.30010604453871
Progress 99.31071049840932
Progress 99.32131495227996
Progress 99.33191940615058
Progress 99.3425238600212
Progress 99.35312831389183
Progress 99.36373276776246
Progress 99.37433722163308
Progress 99.38494167550371
Progress 99.39554612937434
Progress 99.40615058324497
Progress 99.41675503711559
Progress 99.4273594909862
Progress 99.43796394485685
Progress 99.44856839872746
Progress 99.45917285259809
Progress 99.46977730646871
Progress 99.48038176033934
Progress 99.49098621420997
Progress 99.5015906680806
Progress 99.51219512195122
Progress 99.52279957582184
Progress 99.53340402969248
Progress 99.54400848356309
Progress 99.55461293743372
Progress 99.56521739130434
Progress 99.57582184517497
Progress 99.5864262990456
Progress 99.59703075291623
Progress 99.60763520678685
Progress 99.61823966065748
Progress 99.6288441145281
Progress 99.63944856839872
Progress 99.65005302226936
Progre

In [ ]:
df_home_add.to_csv('home_data.csv')

Great, so now we got the features for the ~9500 listings. Let's see how many features we ended up with and find out how much cleaning we have to do.

In [51]:
df_home_add.head()

,adresse,andenmaegler,boligOrGrundAreal,boligurl,city,ejendomstypePrimaerNicename,isNew,lat,lng,openHouseEndDate,...,Ydre mur,Ydre murværk,Ydremur,Ydremure,Ydrevægge,_________________________,købers brug af ejendomme,varmeinstallation:,Øvrige oplysninger,Øvrige oplysninger:
0,"A.D. Jørgensens Vej 75, 2. 1.",False,35.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2019-10-27T11:20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Holger Danskes Vej 14, 3. th.",False,46.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Lyøvej 5, st.. tv.",False,60.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"H. Schneekloths Vej 13, 5. th.",False,56.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,"Howitzvej 61, 3. th.",False,67.0,https://home.dk/boligkatalog/frederiksberg/200...,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2019-11-03T15:20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
cols = df_home_add.columns.tolist()
cols

['adresse',
 'andenmaegler',
 'boligOrGrundAreal',
 'boligurl',
 'city',
 'ejendomstypePrimaerNicename',
 'isNew',
 'lat',
 'lng',
 'openHouseEndDate',
 'openHouseStartDate',
 'overskrift2',
 'postal',
 'price',
 'sagsnummer',
 '****hvis boligen er udlej',
 'Aconto forbrug  pr. måned',
 'Adgangsvej',
 'Adresse',
 'Afløb',
 'Afløbsforhld',
 'Afløbsforhold',
 'Afløbsforhold:',
 'Afstand indkøb',
 'Afstand til indkøb',
 'Afstand til mariager fjor',
 'Afstand til off. transport',
 'Afstand til skole',
 'Afstand til skov',
 'Afstand til strand',
 'Afstand til vand',
 'Afstande',
 'Alarm',
 'Altan',
 'Altan:',
 'Alternative energikilder',
 'Andelboligforenings hjemm',
 'Andelsboligforening',
 'Andelsforening',
 'Andet',
 'Andre bygningsændringer',
 'Anetenneforhold',
 'Anlægsarbejder/påbud',
 'Antal plan',
 'Antal rum',
 'Antal toiletter',
 'Anten.forh.',
 'Antenn.forh.',
 'Antenne',
 'Antenne & internet',
 'Antenne forh.',
 'Antenne forhold',
 'Antenne forhold:',
 'Antenne og internet',
 'A

Alright, so knowing Danish (and you just have to trust me on that one), there are some very odd features like 'Byplansvedtægt', which translate to urban plan regulation, or 'Centralstøvsuger' which means central vacuum cleaner. Furthermore we have a lot of different features that are probably the same like 'Altan' and 'Altan:' (balcony).

So the first two tasks include getting rid of the weird stuff and merging columns.

We have a lot of columns, so let's get to it. To keep an overview let's review a subset at a time.

In [364]:
dropped_cols = []

In [365]:
df_home_add[cols[0:10]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 10 columns):
adresse                        9430 non-null object
andenmaegler                   9430 non-null bool
boligOrGrundAreal              9418 non-null float64
boligurl                       9430 non-null object
city                           9429 non-null object
ejendomstypePrimaerNicename    9430 non-null object
isNew                          9430 non-null bool
lat                            9055 non-null float64
lng                            9055 non-null float64
openHouseEndDate               1504 non-null object
dtypes: bool(2), float64(3), object(5)
memory usage: 1001.5+ KB


We can drop 'andenmaegler' and boligurl now as we don't need them anymore. The open house end date is only practical information as missing values don't indicate whether there's an open house or not, so that should be dropped as well.
The rest we should keep and no need for merging.

In [366]:
dropped_cols.extend(['andenmaegler','boligurl','openHouseEndDate'])

In [367]:
df_home_add[cols[10:20]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 10 columns):
openHouseStartDate           1504 non-null object
overskrift2                  8660 non-null object
postal                       9430 non-null int64
price                        9430 non-null object
sagsnummer                   9430 non-null object
****hvis boligen er udlej    0 non-null object
Aconto forbrug  pr. måned    205 non-null object
Adgangsvej                   0 non-null object
Adresse                      0 non-null object
Afløb                        1 non-null object
dtypes: int64(1), object(9)
memory usage: 1.1+ MB


overskrift2 is the second heading which doesn't really say anything about the listing, so that should be dropped along with the openHouseStartDate, the weird **** column that doesn't contain data anyway, the Adgangsvej (access way) and the Adresse since we already have that. Afløb (drainage) was one of the columns with multiple forms, so let's dive into that.

In [368]:
dropped_cols.extend(['openHouseStartDate','overskrift2','****hvis boligen er udlej','Adgangsvej','Adresse'])

In [369]:
df_home_add[cols[19:32]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 13 columns):
Afløb                         1 non-null object
Afløbsforhld                  0 non-null object
Afløbsforhold                 8 non-null object
Afløbsforhold:                0 non-null object
Afstand indkøb                1 non-null object
Afstand til indkøb            1771 non-null object
Afstand til mariager fjor     1 non-null object
Afstand til off. transport    1451 non-null object
Afstand til skole             1503 non-null object
Afstand til skov              138 non-null object
Afstand til strand            1 non-null object
Afstand til vand              398 non-null object
Afstande                      0 non-null object
dtypes: object(13)
memory usage: 1.3+ MB


Okay, so not very many rows have them anyway, so let's drop everything related to Afløb.

In [370]:
dropped_cols.extend(['Afløb','Afløbsforhld','Afløbsforhold','Afløbsforhold:'])

Afstand indkøb and Afstand til indkøb (distance to grocery shopping) is the same so they should be merged.

Afstand til mariager fjor (specific place in nothern Denmark), Afstand til strand (beach) and Afstande should be dropped.

In [371]:
dropped_cols.extend(['Afstand til mariager fjor','Afstand til strand','Afstande'])

In [372]:
df_home_add[cols[32:50]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 18 columns):
Alarm                        0 non-null object
Altan                        263 non-null object
Altan:                       2 non-null object
Alternative energikilder     0 non-null object
Andelboligforenings hjemm    0 non-null object
Andelsboligforening          0 non-null object
Andelsforening               1 non-null object
Andet                        1 non-null object
Andre bygningsændringer      0 non-null object
Anetenneforhold              1 non-null object
Anlægsarbejder/påbud         0 non-null object
Antal plan                   1577 non-null object
Antal rum                    1941 non-null object
Antal toiletter              1486 non-null object
Anten.forh.                  0 non-null object
Antenn.forh.                 0 non-null object
Antenne                      340 non-null object
Antenne & internet           0 non-null object
dtypes: object(18)
memory usage: 

Alarm, Alternative energikilder, Andelboligforenings hjemm, Andelsboligforening, Andelsforening, Andet, Andre bygningsændringer, Anlægsarbejder/påbud should all be dropped

In [373]:
dropped_cols.extend(['Alarm', 'Alternative energikilder', 
                     'Andelboligforenings hjemm', 'Andelsboligforening', 
                     'Andelsforening', 'Andet', 'Andre bygningsændringer', 
                     'Anlægsarbejder/påbud'])

Altan and Altan: should be merged. Along with anything related to antenna.

In [374]:
df_home_add[cols[50:80]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 30 columns):
Antenne forh.                6 non-null object
Antenne forhold              19 non-null object
Antenne forhold:             0 non-null object
Antenne og internet          4 non-null object
Antenne og it                0 non-null object
Antenne- og internetforho    6 non-null object
Antenne/bredbånd             1 non-null object
Antenne/fibernet             0 non-null object
Antenne/internet             1 non-null object
Antenne/internet:            2 non-null object
Antenne/parabol              1 non-null object
Antenne/tv                   0 non-null object
Antenne/tv/internet          1 non-null object
Antenne1                     1 non-null object
Antenne:                     31 non-null object
Antenne: (enhver udgift h    0 non-null object
Antenneforbindelse           0 non-null object
Antenneforening              0 non-null object
Antenneforh.                 8 non-null obje

The Antenne stuff needs to be investigated on it's own and to be merged into a single (or a few) column(s).

Arealer (areas) can be dropped along with Bad (bath), Badeværelse (bathroom) and Bebyggelse.

In [375]:
dropped_cols.extend(['Arealer', 'Bad', 'Badeværelse', 'Bebyggelse:'])

In [376]:
df_home_add[cols[80:100]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 20 columns):
Bebyggelsens højde                                                                     0 non-null object
Bebyggelses højde                                                                      2 non-null object
Bebyggelsesprocent                                                                     9 non-null object
Bebyggelsesprocent:                                                                    0 non-null object
Benyttelse                                                                             0 non-null object
Beplantning                                                                            0 non-null object
Bevaringskategori                                                                      4 non-null object
Bevaringsværdi                                                                         16 non-null object
Bevaringsværdi:                                    

Everything but Boligarea (estate area) and the weird looking Brutto/Netto columns can be droppped.

In [377]:
dropped_cols.extend(['Bebyggelsens højde', 'Bebyggelses højde',
                     'Bebyggelsesprocent', 'Bebyggelsesprocent:',
                     'Benyttelse', 'Beplantning', 'Bevaringskategori',
                     'Bevaringsværdi', 'Bevaringsværdi:', 'Bevaringsværdig',
                     'Boligydelse pr. måned', 'Bopælspligt:', 'Bortforpagtning:', 
                     'Brugsret til have', 'Brugsret til kælderrum', 'Brændeovn'])

In [378]:
df_home_add[cols[100:120]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 20 columns):
Bugsret til                0 non-null object
Busforbindelser            0 non-null object
Bygepligt:                 1 non-null object
Byggehøjde                 0 non-null object
Byggemodning               0 non-null object
Byggepligt                 25 non-null object
Byggepligt:                0 non-null object
Byggeår                    1945 non-null object
Byggeår / ombygning        0 non-null object
Byggeår /om- tilbygning    0 non-null object
Bylaug                     1 non-null object
Byplansvedtægt:            0 non-null object
Byplanvedtægt              0 non-null object
Bøbelsespligt              0 non-null object
Børnehave                  0 non-null object
Børnehuset bulderby        0 non-null object
Børnehuset lille ørholm    0 non-null object
Carport og udhus           0 non-null object
Centralstøvsuger           0 non-null object
Connection timed out       0 non-

Let's only keep the Byggeår (year of construction).

In [379]:
dropped_cols.extend(cols[100:107])
dropped_cols.extend(cols[108:120])

In [380]:
df_home_add[cols[120:140]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 20 columns):
Cykelkælder                 1 non-null object
Cykelparkering              17 non-null object
Dagenstution                0 non-null object
Daginstitution              10 non-null object
Dagninstitution             0 non-null object
Dagsinstitution             2 non-null object
Dagsinstution               0 non-null object
Depositum                   205 non-null object
Depot til lejligheden       1 non-null object
Depotrum                    11 non-null object
Depotrum:                   2 non-null object
Distriktsskole              1 non-null object
Diverse                     0 non-null object
Dør:                        0 non-null object
Døre indv.                  4 non-null object
Dørtelefon                  53 non-null object
Dørtelefon:                 3 non-null object
Egen parkeringsplads        0 non-null object
Ejendommen kan overtages    0 non-null object
Ejendommens 

All of these can be dropped.

In [381]:
dropped_cols.extend(cols[120:140])

In [382]:
df_home_add[cols[140:180]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 40 columns):
Ejendomsstatus               1 non-null object
Ejendomstype                 0 non-null object
Ejendomsværdi i kr.          1824 non-null object
Ejerforening                 9 non-null object
Ejerforening:                2 non-null object
Ejerforeningen               7 non-null object
Ejerforeningens hjemmesid    0 non-null object
Ejerlaug hjemmeside          0 non-null object
Ejerudgift pr. md.           1825 non-null object
Ekstra areal                 1 non-null object
El                           25 non-null object
El - tilslutningsbidrag      1 non-null object
El stik bidrag               0 non-null object
El:                          1 non-null object
Elektricitet                 0 non-null object
Elevator                     2 non-null object
Elevator i opgang:           2 non-null object
Elinstallationer             1 non-null object
Energi-optimeringer          0 non-null

Let's keep the Ejendomsværdi i kr. (value of property), Etage (floor) and Energimærke (energy rating).

In [383]:
dropped_cols.extend(cols[140:142])
dropped_cols.extend(cols[143:160])
dropped_cols.extend(cols[161:164])
dropped_cols.extend(cols[165:180])

In [384]:
df_home_add[cols[180:210]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 30 columns):
Forkøbsret/venteliste:       0 non-null object
Forpagtning                  1 non-null object
Forudbetalt leje             205 non-null object
Forurening                   11 non-null object
Forurening:                  0 non-null object
Forventet indflytning        2 non-null object
Forventet overtagelse        3 non-null object
Forældrekøb                  0 non-null object
Forældrekøb muligt           1 non-null object
Forældrekøb muligt:          0 non-null object
Forældrekøb:                 1 non-null object
Fredskov:                    1 non-null object
Fundament                    31 non-null object
Fundament:                   27 non-null object
Fælles faciliteter           1 non-null object
Fælles parkeringsplads       0 non-null object
Fælles vaskeri               0 non-null object
Fællesantenne                42 non-null object
Fællesantenne:               0 non-null 

The Forudbetalt leje means prepaid rent - maybe some rentals snuck their way into our dataset! This should be investigated. But otherwise all of the above features can be dropped.

In [385]:
dropped_cols.extend(cols[180:210])

In [386]:
df_home_add[cols[210:233]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 23 columns):
Garage/carport               0 non-null object
Garanti                      0 non-null object
Genvex                       0 non-null object
Gf.forening                  15 non-null object
Gf.forening uden medlemsp    1 non-null object
Gr. ejerforening             0 non-null object
Gr. forening                 1 non-null object
Gr.ejerforening              0 non-null object
Gr.forening                  6 non-null object
Grund                        0 non-null object
Grund-/ejerforening          0 non-null object
Grundareal                   1531 non-null object
Grundeejerforening           0 non-null object
Grundejer-/vejforening       0 non-null object
Grundejerfor.                0 non-null object
Grundejerforening            209 non-null object
Grundejerforening / vejla    0 non-null object
Grundejerforening:           42 non-null object
Grundejerforeningen          0 non-null

Let's keep the Grundareal (ground area) and drop the rest - the 

In [387]:
dropped_cols.extend(cols[210:221])
dropped_cols.extend(cols[222:233])

In [388]:
df_home_add[cols[233:300]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 67 columns):
Gulv                         37 non-null object
Gulv i køkken                0 non-null object
Gulv:                        0 non-null object
Gulvarme                     1 non-null object
Gulvbelægning                0 non-null object
Gulvbelægning :              0 non-null object
Gulvbelægninger              16 non-null object
Gulve                        793 non-null object
Gulve yderligere             1 non-null object
Gulve/lofter                 2 non-null object
Gulve/lofter:                0 non-null object
Gulve:                       51 non-null object
Gulvvarme                    2 non-null object
Gulvvarme, rum               0 non-null object
Gulvvarme, rum:              0 non-null object
Gulvvarme:                   0 non-null object
Gårdmiljø                    31 non-null object
Gårdmiljø/have               0 non-null object
Gårdsmiljø/have              1 non-null 

Anything related to Gulve (flooring) we're gonna keep (and merge) and anything related to the Internet can possibly be merged with the antenna columns, so we'll keep those as well for now. The Heraf grundværdi i kr. is also refering to the estate value so that can maybe also be merged with the previous column. Indkøb (shopping) could also be realted to earlier columns. Everything else can be dropped.

In [389]:
dropped_cols.extend(cols[249:258])
dropped_cols.extend(cols[259:278])
dropped_cols.extend(cols[279:282])
dropped_cols.extend(cols[289:300])

In [390]:
df_home_add[cols[300:350]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 50 columns):
Isolering - lofter           0 non-null object
Isolering - lofter & skun    0 non-null object
Isolering - tilbygning       0 non-null object
Isolering - udestue          0 non-null object
Isolering - vægge            0 non-null object
Isolering - vægge i opr.     0 non-null object
Isolering - vægge i tilby    0 non-null object
Isolering - vægge kælder     0 non-null object
Isolering - vægge stuepla    0 non-null object
Isolering - ydervægge        0 non-null object
Isolering gulv               0 non-null object
Isolering gulv - kælder      0 non-null object
Isolering gulve              0 non-null object
Isolering iflg. energimær    0 non-null object
Isolering iflg. sælger:      1 non-null object
Isolering iflg. sælger: l    1 non-null object
Isolering iflg. sælgers o    0 non-null object
Isolering iht. ejendommen    0 non-null object
Isolering iht. energimærk    0 non-null object

Kloak (sewers) seem like something to keep, alo the kabel-tv/internet stuff is also related to the antenna columns. The kontantpris is the listingprice, which we already have and the Kvm. pris is the price per squaremeter, which we can derive. So let's keep the sewer stuff and cable-tv/internet and drop the rest.

In [391]:
dropped_cols.extend(cols[300:328])
dropped_cols.extend(cols[340:350])

In [392]:
df_home_add[cols[350:400]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 50 columns):
Kælderareal                  439 non-null object
Kælderrum                    9 non-null object
Kælderrum / loftrum          0 non-null object
Kælderrum/loftrum            0 non-null object
Køb af 5 grunde i udstykn    0 non-null object
Køb af de 3 nederste grun    0 non-null object
Køb af de 4 første grunde    0 non-null object
Købesumsfordeling            0 non-null object
Købspris                     22 non-null object
Køkken                       14 non-null object
Køkken og inventar           0 non-null object
Køkken og invetar            0 non-null object
Landbrugspligt               0 non-null object
Landbylaug                   0 non-null object
Landsbylaug & aktivitetsf    1 non-null object
Legeplads                    1 non-null object
Leje pr. måned               205 non-null object
Lejeforhold                  38 non-null object
Lejeindtægt                  2 non-null

Let's keep the stuff about kælder (basement) and Loft (attic). The Købspris is the listing price (again) so that can be discared.

In [393]:
dropped_cols.extend(cols[352:372])
dropped_cols.extend(cols[383:400])

In [394]:
df_home_add[cols[400:450]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 50 columns):
Modrerniseringer             0 non-null object
Motorvej                     0 non-null object
Mulighed for bredbånd        0 non-null object
Mulighed for udlejning       0 non-null object
Mulighed for udlejning af    5 non-null object
Mur                          1 non-null object
Muret/isoleret anneks        0 non-null object
Muret/isoleret garage        0 non-null object
Murværk                      0 non-null object
Nedgravet olietank           1 non-null object
Nedsænket lofter             1 non-null object
Nlærmeste skole              1 non-null object
Næmeste skole                0 non-null object
Nærmeste børnehave           0 non-null object
Nærmeste folkeskole          15 non-null object
Nærmeste skole               8 non-null object
Nærmeste skole:              1 non-null object
Offentlig transport          1 non-null object
Olietank                     2 non-null objec

Overtagelse is just the acquisition date, which doesn't carry that much info. The prisudvikling is the price development, which might be interesting.

In [395]:
dropped_cols.extend(cols[400:447])
dropped_cols.extend(cols[448:450])

In [396]:
df_home_add[cols[450:550]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 100 columns):
Pulterrrum:                  1 non-null object
Pulterrum                    27 non-null object
Pulterrum:                   2 non-null object
Sagsnr.                      2052 non-null object
Samlet bebygget areal        0 non-null object
Sekundær varmekilde          0 non-null object
Sekundær varmkilde           0 non-null object
Septictank                   0 non-null object
Skole                        248 non-null object
Skole/tilhørsforhold         0 non-null object
Skole:                       9 non-null object
Skoledistrikt                11 non-null object
Skoledistrikt:               24 non-null object
Skoleforhold                 0 non-null object
Skoletihørsforhold           1 non-null object
Skoletilhørsforhold          46 non-null object
Skoletilhørsforhold:         0 non-null object
Skov                         0 non-null object
Skur                         0 non-n

Skole (school) might be related to a previous column, so let's keep that. Sagsnr. is just the id and we already have that in another column. Tilbehør (accessories) might be interesting, so let's keep that. Udbetaling is the down payment and is derived from the listing price, so let's drop everything besides the skole columns and the Tilbehør.

In [397]:
dropped_cols.extend(cols[450:458])
dropped_cols.extend(cols[467:507])
dropped_cols.extend(cols[508:550])

In [398]:
df_home_add[cols[550:600]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 50 columns):
Vand - varme  - kloak      0 non-null object
Vand - vej  - kloak        0 non-null object
Vand - vej - kloak         1 non-null object
Vand / vej / kloak         4 non-null object
Vand/vej                   11 non-null object
Vand/vej/kloak             42 non-null object
Vand/vej/kloak:            6 non-null object
Vand/vej:                  0 non-null object
Vand:                      5 non-null object
Vandforsyning              6 non-null object
Vandforsyning :            0 non-null object
Vandværk                   34 non-null object
Varemneinstallation        1 non-null object
Varme                      77 non-null object
Varme installation         1 non-null object
Varme og vand              0 non-null object
Varme, el og vand          0 non-null object
Varme/vand                 0 non-null object
Varme:                     3 non-null object
Varmeforhold               3 non-

Anything related to Varme (heat) we're gonna keep for now, but drop the rest.

In [399]:
dropped_cols.extend(cols[550:562])
dropped_cols.extend(cols[591:600])

In [400]:
df_home_add[cols[600:]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9430 entries, 0 to 51069
Data columns (total 54 columns):
Vej :                        1 non-null object
Vej:                         13 non-null object
Vejfond                      0 non-null object
Vejforening                  0 non-null object
Vejlaug                      1 non-null object
Venteliste                   0 non-null object
Ventilation                  1 non-null object
Vicevært                     75 non-null object
Vicevært/ejendomsservice     1 non-null object
Vicevært:                    6 non-null object
Villæn er tegnet af          0 non-null object
Vinder                       0 non-null object
Vindue                       0 non-null object
Vinduer                      978 non-null object
Vinduer & døre               0 non-null object
Vinduer - resterende vind    0 non-null object
Vinduer - sydvendt facade    0 non-null object
Vinduer / yderdøre           0 non-null object
Vinduer / årgang             1 non-null ob

Vinduer (windows), vurderingsår (year of value assesment) and Ydermur (outer wall/exterior) we're gonna keep, and the rest is to be dropped.

In [401]:
dropped_cols.extend(cols[600:612])
dropped_cols.extend(cols[625:628])
dropped_cols.extend(cols[629:630])
dropped_cols.extend(cols[649:])

Let's have a look at the columns we're gonna drop:

In [402]:
dropped_cols

['andenmaegler',
 'boligurl',
 'openHouseEndDate',
 'openHouseStartDate',
 'overskrift2',
 '****hvis boligen er udlej',
 'Adgangsvej',
 'Adresse',
 'Afløb',
 'Afløbsforhld',
 'Afløbsforhold',
 'Afløbsforhold:',
 'Afstand til mariager fjor',
 'Afstand til strand',
 'Afstande',
 'Alarm',
 'Alternative energikilder',
 'Andelboligforenings hjemm',
 'Andelsboligforening',
 'Andelsforening',
 'Andet',
 'Andre bygningsændringer',
 'Anlægsarbejder/påbud',
 'Arealer',
 'Bad',
 'Badeværelse',
 'Bebyggelse:',
 'Bebyggelsens højde',
 'Bebyggelses højde',
 'Bebyggelsesprocent',
 'Bebyggelsesprocent:',
 'Benyttelse',
 'Beplantning',
 'Bevaringskategori',
 'Bevaringsværdi',
 'Bevaringsværdi:',
 'Bevaringsværdig',
 'Boligydelse pr. måned',
 'Bopælspligt:',
 'Bortforpagtning:',
 'Brugsret til have',
 'Brugsret til kælderrum',
 'Brændeovn',
 'Bugsret til',
 'Busforbindelser',
 'Bygepligt:',
 'Byggehøjde',
 'Byggemodning',
 'Byggepligt',
 'Byggepligt:',
 'Byggeår / ombygning',
 'Byggeår /om- tilbygning',

In [1054]:
df_clean = df_home_add.copy()

In [1055]:
df_clean.drop(columns = dropped_cols, inplace = True)

In [409]:
df_clean.to_csv('home_data_clean.csv')

First of let's look at the rentals that might have snuck into the dataset and let's drop them.

In [1056]:
df_clean.ejendomstypePrimaerNicename.unique()

array(['Ejerlejlighed', 'Andelsbolig', 'Rækkehus', 'Lejebolig',
       'Villalejlighed', 'Villa', 'Fritidshus', 'Helårsgrund',
       'Landejendom', 'Fritidsgrund'], dtype=object)

In [1057]:
df_clean[df_clean.ejendomstypePrimaerNicename == 'Lejebolig'].head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Altan,Altan:,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Anten.forh.,Antenn.forh.,Antenne,Antenne & internet,Antenne forh.,Antenne forhold,Antenne forhold:,Antenne og internet,Antenne og it,Antenne- og internetforho,Antenne/bredbånd,Antenne/fibernet,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv,Antenne/tv/internet,Antenne1,Antenne:,Antenne: (enhver udgift h,Antenneforbindelse,Antenneforening,Antenneforh.,Antenneforh.:,Antenneforhold,Antenneforhold og interne,Antenneforhold.,Antenneforhold:,Antennen,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulv i køkken,Gulv:,Gulvarme,Gulvbelægning,Gulvbelægning :,Gulvbelægninger,Gulve,Gulve yderligere,Gulve/lofter,Gulve/lofter:,Gulve:,Gulvvarme,"Gulvvarme, rum","Gulvvarme, rum:",Gulvvarme:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/antenne:,Internet/tv,Internet:,Internetforbindelse,Kabel-tv,Kabel-tv/internet,Kabeltv / internet,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kloakering :,Kloaksepareret,Kloakseparering,Kloark,Kælderareal,Kælderrum,Loft,Loft / vægge / gulve,Loft:,Loftbeklædning,Loftbeklædninmg,Lofte,Lofter,Lofter :,Lofter og vægge,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole/tilhørsforhold,Skole:,Skoledistrikt,Skoledistrikt:,Skoleforhold,Skoletihørsforhold,Skoletilhørsforhold,Skoletilhørsforhold:,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme og vand,"Varme, el og vand",Varme/vand,Varme:,Varmeforhold,Varmeforsyning,Varmeforsyning :,Varmeforsyning primær,Varmeforsyning sekundær,Varmeinst.,Varmeinstalation,Varmeinstallaion,Varmeinstallation,Varmeinstallation:,Varmeinstallationer,Varmeinstalletion:,Varmeinstillation,Varmekilde,Varmekilde hus 2:,Varmekilde primær,Varmekilde primær:,Varmekilde sekundær,Varmekilde sekundær:,Varmelplan,Varmeplan,Varmeplan:,Vindue,Vinduer,Vinduer & døre,Vinduer - resterende vind,Vinduer - sydvendt facade,Vinduer / yderdøre,Vinduer / årgang,Vinduer mv.,Vinduer og døre,Vinduer og yderdøre,Vinduer/rammer,Vinduer/yderdøre,Vinduer:,Vurderingsår,Ydemur,Yder vægge,Ydermur,Ydermur :,Ydermur bygning 2,Ydermur:,Ydermure,Ydermure:,Ydermyr:,Ydervæg,Ydervæg:,Ydervægge,Ydervægge:,Ydervæggens materiale,Ydre mur,Ydre murværk,Ydremur,Ydremure,Ydrevægge
430,"Teglværksgade 27B, 3. mf.",97.0,København Ø,Lejebolig,False,55.707112,12.558923,2100,0,1770000316,1.150 kr.,NaN,800 m,500 m,450 m,NaN,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97 m2,NaN,NaN,NaN,2017,NaN,A15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
475,"Teglværksgade 27B, St. Tv.",102.0,København Ø,Lejebolig,False,55.707010,12.559074,2100,0,1770000309,1.050 kr.,NaN,400 m,700 m,750 m,NaN,NaN,NaN,NaN,NaN,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101 m2,NaN,NaN,NaN,2020,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [1058]:
df_clean.drop(df_clean[df_clean.ejendomstypePrimaerNicename == 'Lejebolig'].index, inplace = True)

Okay, let's check if we have any columns that only contain NaN and drop those.

In [1059]:
df_clean.dropna(axis = 1, how = 'all', inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Altan,Altan:,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:,Vinduer,Vinduer og yderdøre,Vinduer:,Vurderingsår,Yder vægge,Ydermur,Ydermur:,Ydermure,Ydervæg,Ydervæg:,Ydervægge,Ydremur
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,Nej,NaN,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,NaN,NaN,NaN,NaN,NaN,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,NaN,NaN,NaN,NaN,NaN,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,Nej,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,NaN,NaN,NaN,NaN,NaN,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Ja,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Alright time to have a look at the columns that we might want to merge (and maybe renaming some of them in English).

In [1060]:
df_clean.columns.to_list()

['adresse',
 'boligOrGrundAreal',
 'city',
 'ejendomstypePrimaerNicename',
 'isNew',
 'lat',
 'lng',
 'postal',
 'price',
 'sagsnummer',
 'Aconto forbrug  pr. måned',
 'Afstand indkøb',
 'Afstand til indkøb',
 'Afstand til off. transport',
 'Afstand til skole',
 'Afstand til skov',
 'Afstand til vand',
 'Altan',
 'Altan:',
 'Anetenneforhold',
 'Antal plan',
 'Antal rum',
 'Antal toiletter',
 'Antenne',
 'Antenne forh.',
 'Antenne forhold',
 'Antenne og internet',
 'Antenne/bredbånd',
 'Antenne/internet',
 'Antenne/internet:',
 'Antenne/parabol',
 'Antenne/tv/internet',
 'Antenne:',
 'Antenneforh.',
 'Antenneforhold',
 'Antenneforhold:',
 'Antennetilslutning',
 'Boligareal',
 'Bredbånd',
 'Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift',
 'Brutto/Netto\r\n                        ekskl. ejerudgift',
 'Byggeår',
 'Ejendomsværdi i kr.',
 'Energimærke',
 'Etage',
 'Grundareal',
 'Gulv',
 'Gulvbelægninger',
 'Gulve',
 'Gulve/lofter',
 'Gulve:',
 'Her

We'll start of with the Balcony:

In [1061]:
df_clean['Altan'].unique()

array(['Nej', nan, 'Ja'], dtype=object)

In [1062]:
df_clean['Altan:'].unique()

array([nan, 'Ja'], dtype=object)

In [1063]:
df_clean[df_clean['Altan:'] == 'Ja']

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Altan,Altan:,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:,Vinduer,Vinduer og yderdøre,Vinduer:,Vurderingsår,Yder vægge,Ydermur,Ydermur:,Ydermure,Ydervæg,Ydervæg:,Ydervægge,Ydremur
338,"Jellingegade 7, 2. tv.",75.0,København Ø,Ejerlejlighed,False,55.706620,12.579088,2100,3.995.000,1260000021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,1,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92 m2,NaN,12.371 / 10.829 kr.,NaN,2010,1.850.000,B,5,NaN,NaN,NaN,NaN,NaN,Plankegulve - Mosaikfliser på badeværelse,415.600,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3%,NaN,NaN,NaN,NaN,NaN,Indesit vaske og tørremaskine (kombi) - årgang...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Termoruder,2018,NaN,NaN,Metalplader,NaN,NaN,NaN,NaN,NaN
339,Vangehusvej 6B,76.0,København Ø,Ejerlejlighed,False,55.720263,12.575785,2100,4.495.000,1290000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ja,Ja,NaN,1,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92 m2,NaN,13.528 / 11.842 kr.,NaN,2007,1.800.000,B,2,NaN,NaN,NaN,NaN,NaN,Parketgulve - Fliser på badeværelse,351.700,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,Siemens tørretumbler - årgang 2007Siemens vask...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Termovinduer,2018,NaN,NaN,Andet materiale,NaN,NaN,NaN,NaN,NaN


Alright the other Altan column contain data for those two instances as well, so let's drop the other column.

In [1064]:
df_clean.drop(columns = 'Altan:', inplace = True)

Let's replace the Ja and Nej with Yes and No.

In [1065]:
df_clean['Altan'].replace('Ja','Yes', inplace = True)
df_clean['Altan'].replace('Nej','No', inplace = True)
df_clean['Altan'].unique()

array(['No', nan, 'Yes'], dtype=object)

Finally, let's rename it:

In [1066]:
df_clean.rename(columns={"Altan": "Balcony"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:,Vinduer,Vinduer og yderdøre,Vinduer:,Vurderingsår,Yder vægge,Ydermur,Ydermur:,Ydermure,Ydervæg,Ydervæg:,Ydervægge,Ydremur
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,NaN,NaN,NaN,NaN,NaN,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,NaN,NaN,NaN,NaN,NaN,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,NaN,NaN,NaN,NaN,NaN,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next let's take a look at the outer wall/exterior

In [1067]:
exterior_cols = df_clean.columns.to_list()[-8:]
exterior_cols

['Yder vægge',
 'Ydermur',
 'Ydermur:',
 'Ydermure',
 'Ydervæg',
 'Ydervæg:',
 'Ydervægge',
 'Ydremur']

In [1068]:
def print_unique_values(df, col_list):
    ''' Get the unique values from each column in the col_list for the dataframe'''
    for col in col_list:
        unique_values = df[col].unique()
        print('{}: {}'.format(col, unique_values))

In [1069]:
print_unique_values(df_clean, exterior_cols)

Yder vægge: [nan 'Mursten']
Ydermur: [nan 'Mursten' 'Pudset' 'Mursten (tegl, kalksten, cementsten)'
 'Betonelementer' 'Træbeklædning' 'Built-up' 'mursten' 'Gasbeton'
 'Beton/Teglsten' 'Fibercement' 'Letbeton' 'Tegl/kalksandsten'
 'Pudset mursten' 'Røde mursten' 'Pudsede' 'Hvidpudsede' 'Gule pudsede'
 'Mursten, pudset' 'Træværk' 'Aqua paneler og Cedertræ' 'Træ'
 'Mursten og træ' 'Beton' 'Mursten / træ' 'Gule sten' 'gule sten'
 'Beklædningstegl i byens farver' 'Skiferbeklædning og listebeklædning'
 'Bindingsværk' 'Mudset mursten og gasbeton' 'Hvidpudset'
 'Pudset - Hvidt - Mursten' 'Pudset, malet' 'Pudset malede mursten'
 'Røde og gule mursten' 'Ydervæg i Træ (gran)' 'Hvide kalk/sandsten'
 'Pudset/bindingsværk' 'Pudsede mursten' 'Mursten, bindingsværk, gasbeton'
 'Mursten og pudsede facader' 'Mursten/letbeton' 'Malet mv.'
 'Hvide kalksandsten' 'Vandskuret og malet' 'Pudset/malet'
 'Vandskuret og malet i 2018' 'Vandskuret/malet' 'Mursten/malet'
 'Gule mursten' 'Mursten - malet' 'Kalksands

In [1070]:
df_clean[exterior_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 8 columns):
Yder vægge    1 non-null object
Ydermur       561 non-null object
Ydermur:      23 non-null object
Ydermure      1 non-null object
Ydervæg       11 non-null object
Ydervæg:      1 non-null object
Ydervægge     7 non-null object
Ydremur       1 non-null object
dtypes: object(8)
memory usage: 598.7+ KB


Seems like the Ydermur column is the one with all the juice.

In [1071]:
def merge_columns(df, col_list, merged_col):
    '''
    Merging columns containing similar information in a dataframe into a specified column.
    col_list is the list of similar columns. merged_col is the column information should be merged into.
    '''
    # Check for conflicting information:
    for col in col_list:
        # Only check if the column is not the column to be merged into
        if col != merged_col:
            # If the column in the list is the only one containing information
            # when it's not null, then merge values into the merged_col
            if ((~df[~df_clean[col].isnull()][col_list].isna()).sum() != 0).sum() == 1:
                df[merged_col].fillna(df[col], inplace = True)
            else:
                print('The column {} has conflicts with other columns'.format(col))

In [1072]:
merge_columns(df_clean, exterior_cols, 'Ydermur')

In [1073]:
df_clean[exterior_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 8 columns):
Yder vægge    1 non-null object
Ydermur       606 non-null object
Ydermur:      23 non-null object
Ydermure      1 non-null object
Ydervæg       11 non-null object
Ydervæg:      1 non-null object
Ydervægge     7 non-null object
Ydremur       1 non-null object
dtypes: object(8)
memory usage: 598.7+ KB


We can now drop all but the Ydermur column.

In [1074]:
df_clean.drop(axis = 1, 
              columns = ['Yder vægge', 'Ydermur:', 
                         'Ydermure', 'Ydervæg', 
                         'Ydervæg:', 'Ydervægge','Ydremur'],
              inplace = True)
df_clean.rename(columns = {"Ydermur": "Exterior"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:,Vinduer,Vinduer og yderdøre,Vinduer:,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,NaN,NaN,NaN,NaN,NaN,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,NaN,NaN,NaN,NaN,NaN,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,NaN,NaN,NaN,NaN,NaN,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's look at Vinduer next.

In [1075]:
window_cols = ['Vinduer', 'Vinduer og yderdøre', 'Vinduer:']
df_clean[window_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 3 columns):
Vinduer                715 non-null object
Vinduer og yderdøre    2 non-null object
Vinduer:               30 non-null object
dtypes: object(3)
memory usage: 266.1+ KB


In [1076]:
print_unique_values(df_clean, window_cols)

Vinduer: [nan 'Termo' 'Forsatsvinduer' 'Enkeltlags med Optoglas' 'Termovinduer'
 'Termo m. lags glas' 'Lavenergi'
 'Energiruder, træ/hvid indvendig. Udvendig alu/komposit mørk farve'
 'Termoruder' 'Termo lavenergi m. 3 lags glas'
 '3-lags, DW-Godkendte, lavenergiruder,' 'Tolags energirude.'
 'Energiglas, termo' 'Etlags glasruder, termo- og energiruder'
 'Termo- og energiruder' 'Termo-/energiruder' 'Energi- og termoruder'
 'Energiruder' 'Energi-termorude' 'Tolags energiruder'
 'Energitermoruder og enkelt lag glas' 'Trelags termoruder'
 'Tolags termoruder, tolags energiruder og trelags termoruder'
 '2-lags termorude' '3-lags energiruder' '2-lags termoruder'
 'Blandede termo og energiglas' 'Termo, enkelt lags' 'Sprossede'
 'Termovinduer og Energivinduer' 'Se energimærke'
 'Termoruder, tolags energiruder' 'Trelags energiruder'
 'Termo / enkeltlags' 'Termovinduer, dobbeltruder og enkelt'
 'Termovinduer m/energiglas' 'Enkeltlags og koblede'
 'Termoruder fra 2017/2018' '2 lags energiruder' 'T

Let's merge into Vinduer.

In [1077]:
merge_columns(df_clean, window_cols, 'Vinduer')

In [1078]:
df_clean[window_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 3 columns):
Vinduer                747 non-null object
Vinduer og yderdøre    2 non-null object
Vinduer:               30 non-null object
dtypes: object(3)
memory usage: 266.1+ KB


In [1079]:
df_clean.drop(axis = 1, 
              columns = ['Vinduer:', 'Vinduer og yderdøre'],
              inplace = True)
df_clean.rename(columns = {"Vinduer": "Windows"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,NaN,NaN,NaN,NaN,NaN,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,NaN,NaN,NaN,NaN,NaN,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,NaN,NaN,NaN,NaN,NaN,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lets look at the varme columns now.

In [1080]:
heat_cols = df_clean.columns.to_list()[-15:-3]
df_clean[heat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 12 columns):
Varemneinstallation    1 non-null object
Varme                  49 non-null object
Varme installation     1 non-null object
Varme:                 3 non-null object
Varmeforhold           2 non-null object
Varmeforsyning         9 non-null object
Varmeinstallation      325 non-null object
Varmeinstallation:     15 non-null object
Varmekilde             6 non-null object
Varmekilde primær      1 non-null object
Varmeplan              26 non-null object
Varmeplan:             2 non-null object
dtypes: object(12)
memory usage: 864.8+ KB


In [1081]:
print_unique_values(df_clean, heat_cols)

Varemneinstallation: [nan 'Fjernvarme']
Varme: [nan 'Fjernvarme, gulvvarme i alle boliger'
 'Generelt gulvvarme med individuel rumregulering fra ejerforeningens egen varmecentral'
 'Fjernvarme' 'Naturgas' 'Naturgas, brændeovn' 'Naturgas og brændeovn'
 ':Fjernvarme og pejs' 'Fjernvarme og brændeovn'
 'Fjernvarme suppleret af brændeovn' 'Naturgas og pejs' 'Pillefyr'
 'Naturgas suppleret af brændeovn'
 'Naturgas suppleret med el-gulvvarme i badeværelse'
 'Oliefyr og brændeovn' 'Træpilleovne og elvarme' 'Oliefyr'
 'Oliefyr og elvarme' 'Centralvarme fra eget anlæg, et-kammer fyr'
 'Fastbrændsel/piller' 'Ingen'
 'Tilslutningsbidrag er betalt  til fjernvarme'
 'Mulighed for fjernvarme - Silkeborg Forsyning' 'Undersøes'
 'Varmepumpe (luft til luft)' 'Elvarme' 'Elvarme + brændeovn'
 'Elvarme og brændeovn']
Varme installation: [nan 'Fjernvarme']
Varme:: [nan 'Naturgas' 'Fjernvarme' 'ingen - betales af køber']
Varmeforhold: [nan 'Oliefyr' 'Pillefyr']
Varmeforsyning: [nan 'Oliefyr' 'Stokerfyr' 'Ol

Let's merge into Varmeisntallation

In [1082]:
merge_columns(df_clean, heat_cols, 'Varmeinstallation')

The column Varmeinstallation: has conflicts with other columns
The column Varmeplan has conflicts with other columns
The column Varmeplan: has conflicts with other columns


In [1083]:
df_clean[~df_clean['Varmeinstallation:'].isnull()][heat_cols]

,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:
3890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,NaN
3899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,NaN
4338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,NaN
5036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elvarme,NaN,NaN,NaN,NaN
5040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elvarme,NaN,NaN,NaN,NaN
5043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jordvarme suppleret af brændeovn,NaN,NaN,NaN,NaN
5213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Naturgas,NaN,NaN,NaN,Naturgas
5223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Varmepumpe fra 2016,NaN,NaN,NaN,NaN
5224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pillefyr,NaN,NaN,NaN,NaN
5225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjenvarme,NaN,NaN,NaN,NaN


Okay, so the Varmeplan: is sometimes conflicting, but is either the same or not adding more information.

In [1084]:
df_clean[~df_clean['Varmeplan'].isnull()][heat_cols]

,Varemneinstallation,Varme,Varme installation,Varme:,Varmeforhold,Varmeforsyning,Varmeinstallation,Varmeinstallation:,Varmekilde,Varmekilde primær,Varmeplan,Varmeplan:
3633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN
3634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nej,NaN
3660,NaN,NaN,NaN,NaN,NaN,NaN,Pillefyr + pejs,NaN,NaN,NaN,Individuel opvarmning,NaN
3661,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,Fjernvarme,NaN
3704,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,Fjernvarme,NaN
3709,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,Fjernvarme,NaN
3714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN
3723,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN,NaN,NaN,Ingen,NaN
3724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingen,NaN
3726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fjernvarme,NaN


Let's merge it in and discard the rest.

In [1085]:
df_clean['Varmeinstallation'].fillna(df_clean['Varmeinstallation:'], inplace = True)
df_clean['Varmeinstallation'].fillna(df_clean['Varmeplan'], inplace = True)
df_clean['Varmeinstallation'].fillna(df_clean['Varmeplan:'], inplace = True)
df_clean[heat_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 12 columns):
Varemneinstallation    1 non-null object
Varme                  49 non-null object
Varme installation     1 non-null object
Varme:                 3 non-null object
Varmeforhold           2 non-null object
Varmeforsyning         9 non-null object
Varmeinstallation      428 non-null object
Varmeinstallation:     15 non-null object
Varmekilde             6 non-null object
Varmekilde primær      1 non-null object
Varmeplan              26 non-null object
Varmeplan:             2 non-null object
dtypes: object(12)
memory usage: 864.8+ KB


In [1086]:
# Remove the merging column
heat_cols.remove('Varmeinstallation')

In [1087]:
heat_cols

['Varemneinstallation',
 'Varme',
 'Varme installation',
 'Varme:',
 'Varmeforhold',
 'Varmeforsyning',
 'Varmeinstallation:',
 'Varmekilde',
 'Varmekilde primær',
 'Varmeplan',
 'Varmeplan:']

In [1088]:
df_clean.drop(axis = 1, columns = heat_cols, inplace = True)
df_clean.rename(columns = {"Varmeinstallation": "Heating"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,Afstand til skole,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,NaN,NaN,NaN,NaN,NaN,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,NaN,NaN,NaN,NaN,NaN,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,NaN,NaN,NaN,NaN,NaN,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,NaN,NaN,NaN,NaN,NaN,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next let's look at school related columns.

In [1089]:
school_cols = ['Skole','Skole:','Skoledistrikt','Skoletihørsforhold','Skoletilhørsforhold','Afstand til skole']
df_clean[school_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 6 columns):
Skole                  100 non-null object
Skole:                 8 non-null object
Skoledistrikt          2 non-null object
Skoletihørsforhold     1 non-null object
Skoletilhørsforhold    46 non-null object
Afstand til skole      1013 non-null object
dtypes: object(6)
memory usage: 465.7+ KB


In [1090]:
print_unique_values(df_clean, school_cols)

Skole: [nan 'Dyveke eller Ørestad' 'Kalvebod Fælled Skole' 'Skolen ved Sundet'
 'Peder Lykke Skolen' 'Skolen i Sydhavn' 'Herstedøster Skole'
 'Søndervangskolen' 'Glostrup skole, Ejby' 'Søndervangsskolen'
 'Glostrup Skole, Ejby' 'Nord vang skole' 'Frydenhøjskolen'
 'Holmegårdsskolen' 'Engstrandskolen' 'Dansborgskolen' 'Præstemoseskolen'
 'Arenaskolen' 'Hedelyskolen' 'Krogårdskolen' 'Mosedeskolen'
 'Kroggårdskolen / Hedelyskolen' 'Katrinedals skolen 450 m'
 'Skovgårdsskolen' 'Fuglsanggårdsskolen' 'Hummeltofteskolen'
 'Kongevejens Skole' 'Bagsværd Skole' 'Gladsaxe Skole' 'Skovbrynet Skole'
 'Skovbrynet skole' 'Gadevang skole/ Hillerødholmsskolen/ Kontiki'
 'Lille Værløse skole' 'Lille Værløse' 'Syvstjerneskolem' 'Solvangskolen'
 'Lyngholmskolen' 'Lyngholm' 'Ådalens skole' 'Ganløse skole'
 'Vor Frue Skole'
 'Sorø Borgerskole, Sorø Privatskole, Sorø Akademi (gymnasium)'
 'Frederiksberg Skole, Sorø privatskole og Sorø Akademi (gymnasium)'
 'Holmegaardskolen afd. Toksværd' 'Vestskolen, Afd Vi

Alright it looks like this is two seperate types of information. One is the name of the nearest school whereas the second is the distiance. We don't have a lot of data regarding the school name, so let's drop those, but before during so, let's extract the distiance from the few, that contain that information in the string in the Skole column.

In [1091]:
df_clean[df_clean.Skole.str.contains('\d', regex=True).fillna(False)][school_cols]

,Skole,Skole:,Skoledistrikt,Skoletihørsforhold,Skoletilhørsforhold,Afstand til skole
1265,Katrinedals skolen 450 m,NaN,NaN,NaN,NaN,450 m
5368,Hørup Skole 0.-9. klasse,NaN,NaN,NaN,NaN,1.300 m
5369,Hørup Skole 0.-9. klasse,NaN,NaN,NaN,NaN,1.300 m
5370,Hørup Skole 0.-9. klasse,NaN,NaN,NaN,NaN,2.600 m
5383,Hørup Centalskole 0-9 klasse,NaN,NaN,NaN,NaN,1.900 m
5384,Hørup Centalskole 0-9 klasse,NaN,NaN,NaN,NaN,1.900 m
8300,"Bjerringbro/Bøgeskovskolen, afstand ca. 450 m.",NaN,NaN,NaN,NaN,450 m
8301,"Bjerringbro, Egeskovskolen (ca. 2,3 km)",NaN,NaN,NaN,NaN,2.300 m
8302,"Bøgeskovskolen, Bjerringbro, ca. 4,5 km",NaN,NaN,NaN,NaN,4.500 m
8303,"Bjerringbro/Bøgeskovskolen, afstand ca. 1400 m.",NaN,NaN,NaN,NaN,1.400 m


Great it's already there!

In [1092]:
school_cols.remove('Afstand til skole')

In [1093]:
df_clean.drop(axis = 1, columns = school_cols, inplace = True)
df_clean.rename(columns = {"Afstand til skole": "DistanceToSchool"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Kloak,Kloak - tilslutningsbidra,Kloak/vand/vej,Kloak:,Kloakering,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1094]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 75 columns):
adresse                                                                                8515 non-null object
boligOrGrundAreal                                                                      8505 non-null float64
city                                                                                   8515 non-null object
ejendomstypePrimaerNicename                                                            8515 non-null object
isNew                                                                                  8515 non-null bool
lat                                                                                    8221 non-null float64
lng                                                                                    8221 non-null float64
postal                                                                                 8515 non-null int64
price                       

Next let's look at the Kloak columns.

In [1095]:
sewer_cols = df_clean.columns.tolist()[-17:-12]
sewer_cols

['Kloak',
 'Kloak - tilslutningsbidra',
 'Kloak/vand/vej',
 'Kloak:',
 'Kloakering']

In [1096]:
df_clean[sewer_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 5 columns):
Kloak                        359 non-null object
Kloak - tilslutningsbidra    1 non-null object
Kloak/vand/vej               2 non-null object
Kloak:                       10 non-null object
Kloakering                   2 non-null object
dtypes: object(5)
memory usage: 399.1+ KB


In [1097]:
print_unique_values(df_clean, sewer_cols)

Kloak: [nan 'Offentlig' 'se salgsopstilling'
 'Afløb til spildevandsforsyningens renseanlæg'
 'Separatkloakeret: spildevand + tag- og overfladevand'
 'Privat / offentlig' 'Offentlig kloak' 'Er betalt' 'Septiktank'
 'Anden type afløb' 'Mekanisk rensning med nedsivningsanlæg'
 'Seperatkloakeret' 'Faxe Forsyning'
 'Tilsluttet spildevandsforsyningens renseanlæg' 'Mekanisk rensning'
 'Afløb til spildevandforsyningens renseanlæg' 'Guldborgsund Forsyning'
 'Lolland Spildevand' 'Lolland Forsyning' 'Lolland'
 'Mekanisk rensning med nedsivningsanlæg med tilladelse'
 'Afløb til samletank' 'Minirenseanlæg med udledning til markdræn'
 'Afløb til spildevand' 'Offenlig'
 'Afløb til spildevandsforsyningen renseanlæg'
 'Afløb til fællesprivat spildevandsanlæg'
 'Mekanisk rensning med privat udledn. dir. til vandløb, sø eller hav'
 'Spildevandforsyningens renseanlæg' 'Fælles privat' 'Off. kloak'
 'Nedsivning til sivedræn' 'afløb til spildevandesforsynings renseanlæg'
 'Afløb til spildevandesforsyningens

Let's merge into the Kloak column.

In [1098]:
merge_columns(df_clean, sewer_cols, 'Kloak')

In [1099]:
sewer_cols.remove('Kloak')

In [1100]:
df_clean.drop(axis = 1, columns = sewer_cols, inplace = True)
df_clean.rename(columns = {"Kloak": "Sewers"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Anetenneforhold,Antal plan,Antal rum,Antal toiletter,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Boligareal,Bredbånd,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Sewers,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,NaN,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35 m2,NaN,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46 m2,NaN,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77 m2,NaN,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,NaN,1,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74 m2,NaN,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's look at the antenna/tv/internet columns next.

In [1101]:
antenna_cols = df_clean.columns.tolist()[22:-35]
antenna_cols.extend(df_clean.columns.tolist()[52:-13])
antenna_cols.append('Anetenneforhold')
antenna_cols.append('Bredbånd')
antenna_cols

['Antenne',
 'Antenne forh.',
 'Antenne forhold',
 'Antenne og internet',
 'Antenne/bredbånd',
 'Antenne/internet',
 'Antenne/internet:',
 'Antenne/parabol',
 'Antenne/tv/internet',
 'Antenne:',
 'Antenneforh.',
 'Antenneforhold',
 'Antenneforhold:',
 'Antennetilslutning',
 'Internet',
 'Internet-/tv-forhold',
 'Internet/antenne',
 'Internet/tv',
 'Internet:',
 'Kabel-tv',
 'Anetenneforhold',
 'Bredbånd']

In [1102]:
print_unique_values(df_clean, antenna_cols)

Antenne: [nan 'Kabel-tv' 'YouSee' 'Individuelt' 'Parknet Kabel-TV individuelt'
 'Kabel-TV - YouSee' 'Individuelt tilslutning' 'Individuel tilslutning'
 'Yousee' 'Kabel TV' 'Kabel-TV via Fibia'
 'Klargjort til kabel-tv, individuel tilslutning' 'TDC hybridnet'
 'YouSees kabelnet – Grundpakken' 'Hybridnet'
 'Individuel tilslutning med mulighed for parknet (etableret i foreningen)'
 'Kabel tv(via ejerforeningen)' 'Mulighed for kabel-tv'
 'Antenneforening Eriksminde' 'Egen' 'Parabol' 'Individuel'
 'Klargjort til kabel-tv individuel tilslutning & betaling' 'Kabeltv'
 'Privat' 'Kabel-TV + fiber iflg. sælger'
 'Kabel-TV/Hybrid jf. administrator besvarelsen'
 'Parabol - mulighed for Yousee' 'Kabel-tv, Yousee' 'Ikke oplyst'
 'Fibernet' 'Kabel tv, sælger har Yousee' 'Veksø Antenneforening'
 'Antenneforening' 'Kabelnet via Veksø Antenneforening' 'Egen antenne'
 'YouSee kabeltv' 'Parabol uden reciever' 'Parabol/internet'
 'Antenne uden receiver' 'Afventer sælger'
 'Egen antenne - Fibernet ved vejen

In [1103]:
df_clean[antenna_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 22 columns):
Antenne                 231 non-null object
Antenne forh.           6 non-null object
Antenne forhold         19 non-null object
Antenne og internet     4 non-null object
Antenne/bredbånd        1 non-null object
Antenne/internet        1 non-null object
Antenne/internet:       2 non-null object
Antenne/parabol         1 non-null object
Antenne/tv/internet     1 non-null object
Antenne:                3 non-null object
Antenneforh.            7 non-null object
Antenneforhold          180 non-null object
Antenneforhold:         3 non-null object
Antennetilslutning      1 non-null object
Internet                29 non-null object
Internet-/tv-forhold    1 non-null object
Internet/antenne        2 non-null object
Internet/tv             4 non-null object
Internet:               1 non-null object
Kabel-tv                5 non-null object
Anetenneforhold         1 non-null object
Bred

Let's merge into Antenne.

In [1104]:
merge_columns(df_clean, antenna_cols, 'Antenne')

The column Antenneforhold has conflicts with other columns
The column Internet has conflicts with other columns


In [1105]:
df_clean[~df_clean['Antenneforhold'].isnull()][antenna_cols]

,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Anetenneforhold,Bredbånd
1438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grundpakke hos YouSee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oplyses af sælger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fællesantenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fællesantenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Egen antenne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Almindelig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2775,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingen oplysning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingen oplysning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sælger har YouSee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,parabol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1106]:
df_clean[~df_clean['Internet'].isnull()][antenna_cols]

,Antenne,Antenne forh.,Antenne forhold,Antenne og internet,Antenne/bredbånd,Antenne/internet,Antenne/internet:,Antenne/parabol,Antenne/tv/internet,Antenne:,Antenneforh.,Antenneforhold,Antenneforhold:,Antennetilslutning,Internet,Internet-/tv-forhold,Internet/antenne,Internet/tv,Internet:,Kabel-tv,Anetenneforhold,Bredbånd
164,Individuelt tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuelt tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yousee,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yousee Kabelnet - Grundpakken,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,YouSees kabelnet – Grundpakken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871,YouSees kabelnet – Grundpakken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Individuel tilslutning,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,Hybridnet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yousee Kabelnet - Grundpakken,NaN,NaN,NaN,NaN,NaN,NaN,NaN


They seem to be containing the same information so let's merge them in where we have NaNs.

In [1107]:
df_clean['Antenne'].fillna(df_clean['Internet'], inplace = True)
df_clean['Antenne'].fillna(df_clean['Antenneforhold'], inplace = True)
df_clean[antenna_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 22 columns):
Antenne                 488 non-null object
Antenne forh.           6 non-null object
Antenne forhold         19 non-null object
Antenne og internet     4 non-null object
Antenne/bredbånd        1 non-null object
Antenne/internet        1 non-null object
Antenne/internet:       2 non-null object
Antenne/parabol         1 non-null object
Antenne/tv/internet     1 non-null object
Antenne:                3 non-null object
Antenneforh.            7 non-null object
Antenneforhold          180 non-null object
Antenneforhold:         3 non-null object
Antennetilslutning      1 non-null object
Internet                29 non-null object
Internet-/tv-forhold    1 non-null object
Internet/antenne        2 non-null object
Internet/tv             4 non-null object
Internet:               1 non-null object
Kabel-tv                5 non-null object
Anetenneforhold         1 non-null object
Bred

In [1108]:
antenna_cols.remove('Antenne')

In [1109]:
df_clean.drop(axis = 1, columns = antenna_cols, inplace = True)
df_clean.rename(columns = {"Antenne": "Connection"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Gulv,Gulvbelægninger,Gulve,Gulve/lofter,Gulve:,Heraf grundværdi i kr.,Indkøb,Sewers,Kælderareal,Kælderrum,Loft,Lofter,Lofter/gulve,Lofter:,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,NaN,NaN,NaN,NaN,NaN,168.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,NaN,NaN,NaN,NaN,NaN,145.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,NaN,NaN,NaN,NaN,NaN,259.100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,NaN,NaN,NaN,NaN,NaN,164.300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's look at loft and gulv (floor and ceiling).

In [1110]:
floor_ceil_cols = ['Loft', 'Lofter', 'Lofter/gulve',
                   'Lofter:','Gulve:', 'Gulve/lofter',
                   'Gulve', 'Gulvbelægninger', 'Gulv']

In [1111]:
df_clean[floor_ceil_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 9 columns):
Loft               9 non-null object
Lofter             440 non-null object
Lofter/gulve       1 non-null object
Lofter:            27 non-null object
Gulve:             23 non-null object
Gulve/lofter       2 non-null object
Gulve              590 non-null object
Gulvbelægninger    16 non-null object
Gulv               24 non-null object
dtypes: object(9)
memory usage: 665.2+ KB


In [1112]:
print_unique_values(df_clean, floor_ceil_cols)

Loft: [nan 'Beton' 'Profilloft / Gips' 'Liste' 'Træ' 'Hvidmalet træ'
 'Ludbehandlet træ' 'Træ og gipsplader' 'Lud rustik']
Lofter: [nan 'Beton' 'Gips' 'Gipslofter' 'Gips og listelofter' 'Gips og træ'
 'Trælofter' 'Gips og beton' 'Betonlofter' 'Gipsplader'
 'Træ og gipslofter' 'Listelofter' 'Gips og pladelofter' 'Gips,Træ' 'Træ'
 'Gips, træ' 'Hvide gipslofter' 'Træ, gips'
 'Betonlofter spartles og males, lys råhvid' 'Træ, Gips' 'Træ, mm' 'Liste'
 'Gips, lister' 'Liste og gips' 'Pudsede lofter' 'Trælofter, gips mm'
 'Pladelofter / gipsplader / pudsede' 'Listelofter / Gipslofter'
 'Gipsplader / panellofter / listelofter' 'Lister og panellofter' 'Lister'
 'hvide' 'Pudset/gips' 'Gips- og træ' 'Pudset loft, træ'
 'Pladelofter/træ/gips' 'Listelofter, gips' 'Laminat/lofter'
 'Puds med stuk' 'Panellofter' 'Træ, pudset loft' 'Pudsede og gips'
 'Træ/gipsplader' 'Gips lofter , listelofter' 'Profilbrædder' 'Gips/træ'
 'Gips mm.' 'Trælofter mv.' 'Træ, gips mv.' 'Gips, træ mv.'
 'Gips/pladeloft' 'Div

Seems like it's two seperate features. Except for the one with both in them.

In [1113]:
df_clean[~df_clean['Gulve/lofter'].isnull()][floor_ceil_cols]

,Loft,Lofter,Lofter/gulve,Lofter:,Gulve:,Gulve/lofter,Gulve,Gulvbelægninger,Gulv
2775,NaN,NaN,NaN,NaN,NaN,"Træ, klinker, linolium/Træ",NaN,NaN,NaN
2776,NaN,NaN,NaN,NaN,NaN,"Træ, Klinker/Træ",NaN,NaN,NaN


We can't really know if they are refering to ceiling or floor, so let's drop this own and split the rest into 2.

In [1114]:
df_clean.drop(axis = 1, columns = 'Gulve/lofter', inplace = True)

In [1115]:
floor_cols = ['Gulve:',  'Gulve', 'Gulvbelægninger', 'Gulv']
ceil_cols = ['Loft', 'Lofter', 'Lofter/gulve', 'Lofter:']

For floor, let's merge into Gulve.

In [1116]:
merge_columns(df_clean, floor_cols, 'Gulve')

In [1117]:
df_clean.drop(axis = 1, columns = floor_cols, inplace = True)
df_clean.rename(columns = {"Gulve": "Floor"}, inplace = True)

For ceiling let's merge into Lofter.

In [1118]:
merge_columns(df_clean, ceil_cols, 'Lofter')

In [1119]:
df_clean.drop(axis = 1, columns = ceil_cols, inplace = True)
df_clean.rename(columns = {"Lofter": "Ceiling"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Indkøb,Sewers,Kælderareal,Kælderrum,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,168.300,NaN,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,145.000,NaN,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,259.100,NaN,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,164.300,NaN,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next let's tackle the distance to shopping.

In [1120]:
shop_cols = ['Afstand indkøb', 'Afstand til indkøb', 'Indkøb']
df_clean[shop_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 3 columns):
Afstand indkøb        1 non-null object
Afstand til indkøb    1227 non-null object
Indkøb                12 non-null object
dtypes: object(3)
memory usage: 266.1+ KB


In [1121]:
print_unique_values(df_clean, shop_cols)

Afstand indkøb: [nan 'Meny i Als ca. 2,5 km - her er også slagter og bager i byen']
Afstand til indkøb: ['1.100  m' '150  m' '650  m' '50  m' nan '100  m' '250  m' '300  m'
 '200  m' '220  m' '750  m' '400  m' '800  m' '10  m' '110  m' '500  m'
 '170  m' '900  m' '700  m' '1.400  m' '350  m' '600  m' '55  m' '450  m'
 '71  m' '60  m' '210  m' '290  m' '230  m' '190  m' '1.000  m' '270  m'
 '240  m' '950  m' '1.300  m' '550  m' '30  m' '260  m' '2.000  m'
 '1.200  m' '850  m' '3.000  m' '1.070  m' '3.500  m' '280  m' '1.500  m'
 '2.300  m' '2.100  m' '2.500  m' '1.700  m' '5.000  m' '1.600  m'
 '4.200  m' '2.200  m' '3.300  m' '6.000  m' '8.000  m' '4.000  m'
 '2.400  m' '2.600  m' '2.700  m' '5.500  m' '5.700  m' '4.300  m'
 '6.800  m' '3.600  m' '120  m' '4.600  m' '3.800  m' '3.400  m'
 '5.900  m' '5.200  m' '1.800  m' '2.800  m' '5.100  m' '1.900  m'
 '4.500  m' '4.400  m' '130  m' '8.500  m' '11.000  m' '3.350  m'
 '3.700  m' '1.250  m' '5.300  m' '3.900  m' '6.400  m' '6.500  m'
 

Two of the columns are a mix of distance and text, let's check them out:

In [1122]:
df_clean[~df_clean['Afstand indkøb'].isnull()]

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Indkøb,Sewers,Kælderareal,Kælderrum,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
8945,Bygvangen 7,1568.0,Ølsted,Fritidsgrund,False,55.893694,12.054161,3310,495.000,1150000099,NaN,"Meny i Als ca. 2,5 km - her er også slagter og...",3.800 m,NaN,NaN,30 m,Ca. 700 mtr. til strandeng,NaN,1,3,1,NaN,54 m2,2.049 / 1.742 kr.,NaN,1976,430.000,NaN,NaN,2.027 m2,118.500,NaN,NaN,NaN,NaN,0%,"Glaskeramisk el-komfur, microbølgeovn, køle-/f...",NaN,Tophængte trærammer med termoglas,2018,NaN


In [1123]:
df_clean[~df_clean['Indkøb'].isnull()]

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,Afstand indkøb,Afstand til indkøb,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Indkøb,Sewers,Kælderareal,Kælderrum,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
6974,"Vallerød Banevej 1, 1.",113.0,Rungsted Kyst,Villalejlighed,False,55.882873,12.521914,2960,3.695.000,1420000135,NaN,NaN,1.300 m,1.300 m,1.400 m,NaN,NaN,NaN,NaN,6,2,NaN,329 m2,13.919 / 12.041 kr.,NaN,1974,1.950.000,D,NaN,2.542 m2,376.000,Rema1000 (1300 meter),NaN,145 m2,NaN,0%,"Induktions bordkogeplade, emhætte, amerikaner ...",NaN,NaN,2018,NaN
8300,"Lille Orebjergvej 4, Rågeleje",102.0,Vejby,Fritidshus,True,56.104423,12.174963,3210,3.595.000,1570000201,NaN,NaN,350 m,875 m,450 m,NaN,NaN,NaN,3,4,2,NaN,113 m2,5.630 / 4.868 kr.,NaN,1957,1.050.000,E,NaN,846 m2,304.900,"Bjerringbro, afstand ca. 350 m.",Offentlig,67 m2,NaN,0%,"Komfur mrk. Voss årg. 2016, emhætte mrk. Voss ...",NaN,NaN,2018,NaN
8301,"Rørstykket 13, Rågeleje",65.0,Vejby,Fritidshus,True,56.085995,12.170301,3210,1.395.000,1570000187,NaN,NaN,1.400 m,1.500 m,2.300 m,NaN,NaN,NaN,2,5,2,NaN,197 m2,6.593 / 5.701 kr.,NaN,1972,1.300.000,C,NaN,1.000 m2,351.000,"Bjerringbro (ca. 1,4 km)",NaN,NaN,NaN,0%,"Komfur mrk. ""Voss"", emhætte mrk. Voss, køle/sv...",NaN,NaN,2018,NaN
8302,"Bjørnevej 8, Vejby Strand",104.0,Vejby,Fritidshus,False,56.080838,12.128951,3210,2.350.000,1570000166,NaN,NaN,5.000 m,150 m,4.500 m,NaN,NaN,NaN,NaN,3,1,NaN,90 m2,2.529 / 2.185 kr.,NaN,1931,500.000,D,NaN,325 m2,150.400,"Bjerringbro, ca. 5 km",NaN,NaN,NaN,0%,"Komfur mrk. Voss, emhætte mrk. Wasco, køle-/fr...",NaN,NaN,2018,NaN
8303,"Stokkebrovej 35, Unnerup",89.0,Vejby,Villa,False,56.071420,12.167860,3210,1.995.000,1570000169,NaN,NaN,750 m,1.100 m,1.400 m,NaN,NaN,NaN,1,4,2,NaN,130 m2,5.526 / 4.779 kr.,NaN,1977,1.150.000,D,NaN,976 m2,329.000,"Bjerringbro, afstand ca. 750 m.",Offentlig,NaN,NaN,0%,"Komfur mrk. Gram årg. ukendt, emhætte mrk. Blo...",NaN,NaN,2018,NaN
8304,Kælderbjerg Park 15,150.0,Vejby,Villa,False,56.066295,12.148465,3210,2.395.000,1690000038,NaN,NaN,1.300 m,2.000 m,800 m,NaN,NaN,NaN,1,4,1,Bjerringbro Kabelnet og fibernet i vejen,114 m2,5.911 / 5.112 kr.,NaN,1992,1.300.000,B,NaN,431 m2,276.000,Bjerringbro ca 1300 m,Offentlig,NaN,NaN,-4%,"Bordkomfur mrk. Siemens årg. 2009, indbygnings...",fjernvarme,NaN,2017,NaN
8306,Grøndyssevej 23,253.0,Vejby,Villa,False,56.067579,12.148008,3210,2.850.000,1570000067,NaN,NaN,1.000 m,1.000 m,2.100 m,NaN,NaN,NaN,1,5,2,Mulighed for Fiberbredbånd med TV-pakke og Kab...,172 m2,7.660 / 6.624 kr.,NaN,1973,1.250.000,C,NaN,1.120 m2,337.000,"Bjerringbro, afstand ca. 1,0 km.",Offentlig / Separat kloakeret,NaN,NaN,0%,"Bordkomfur mrk. Siemens årg. 2014, indbygnings...",Fjernvarme,NaN,2018,NaN
8307,"Lundeskrænten 13, Holløselund",89.0,Vejby,Fritidshus,False,56.068193,12.113032,3210,1.895.000,1690000020,NaN,NaN,250 m,250 m,200 m,NaN,500 m,NaN,NaN,NaN,NaN,NaN,NaN,1.165 / 926 kr.,NaN,NaN,NaN,E,NaN,1.334 m2,NaN,"Tange, afstand ca. 250 m./Bjerringbro, afst. c...",NaN,NaN,NaN,0%,NaN,NaN,NaN,2018,NaN
8308,"Kongsholmvej 2B, Kolsbæk",221.0,Vejby,Villa,False,56.080840,12.178714,3210,3.495.000,157-03163,NaN,NaN,7.000 m,100 m,6.000 m,NaN,NaN,NaN,2,6,NaN,NaN,242 m2,6.593 / 5.700 kr.,NaN,1860,2.300.000,D,NaN,17.379 m2,345.000,"Bjerringbro/Ulstrup, ca. 7 km",NaN,NaN,NaN,-8%,"Komfur, køleskab, emhætte samt opvaskemaskine.",NaN,NaN,2017,NaN
8309,"Plovmandsvang 45, Rågeleje",89.0,Vejby,Fritidshus,False,56.079997,12.152388,3210,1.595.000,1570000025,NaN,NaN,1.800 m,1.900 m,2.700 m,NaN,NaN,NaN,2,6,2,Ja og fibernet i vejen,198 m2,11.603 / 10.037 kr.,NaN,1988,1.900.000,C,NaN,1.020 m2,283.900,"Bjerringbro, afstand ca. 1,8 km",Offentlig,70 m2,NaN,0%,Bordkomfur - in

In both cases we have the distance in the Afstand til indkøb column already.

In [1124]:
shop_cols.remove('Afstand til indkøb')

In [1125]:
df_clean.drop(axis = 1, columns = shop_cols, inplace = True)
df_clean.rename(columns = {"Afstand til indkøb": "DistanceToShopping"}, inplace = True)

In [1126]:
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,DistanceToShopping,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Sewers,Kælderareal,Kælderrum,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,168.300,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,145.000,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,259.100,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,164.300,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's merge the Brutto/Netto owner's fee columns.

In [1127]:
owner_fee_col = df_clean.columns.tolist()[22:24]
owner_fee_col

['Brutto/Netto\r\n                        ?\n\r\n                        ekskl. ejerudgift',
 'Brutto/Netto\r\n                        ekskl. ejerudgift']

In [1128]:
df_clean[owner_fee_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 2 columns):
Brutto/Netto
                        ?

                        ekskl. ejerudgift    1266 non-null object
Brutto/Netto
                        ekskl. ejerudgift                                2 non-null object
dtypes: object(2)
memory usage: 199.6+ KB


In [1129]:
print_unique_values(df_clean, owner_fee_col)

Brutto/Netto
                        ?

                        ekskl. ejerudgift: ['5.249  / 4.628  kr.' '8.313  / 7.331  kr.' '10.446  / 9.212  kr.' ...
 '13.513  / 11.649  kr.' '14.667  / 12.644  kr.' '16.974  / 14.472  kr.']
Brutto/Netto
                        ekskl. ejerudgift: [nan '30.112  / 26.287  kr.' '2.832  / 2.403  kr.']


Let's merge into the first one and then we should probably split the merged column into two columns as well.

In [1130]:
merge_columns(df_clean, owner_fee_col, owner_fee_col[0])

Let's create two new columns for. One called NetOwnerFee and one called GrossOwnerFee. The former should contain the price before the / and the latter the price after.

In [1131]:
new_cols = df_clean[owner_fee_col[0]].str.split('/',n = 1, expand = True)
df_clean['NetOwnerFee'] = new_cols[0]
df_clean['GrossOwnerFee'] = new_cols[1]
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,DistanceToShopping,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Brutto/Netto ? ekskl. ejerudgift,Brutto/Netto ekskl. ejerudgift,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Sewers,Kælderareal,Kælderrum,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior,NetOwnerFee,GrossOwnerFee
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,5.249 / 4.628 kr.,NaN,1991,780.000,B,2,NaN,168.300,NaN,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN,5.249,4.628 kr.
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,8.313 / 7.331 kr.,NaN,1885,1.100.000,D,3,NaN,145.000,NaN,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN,8.313,7.331 kr.
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,10.446 / 9.212 kr.,NaN,1988,1.600.000,C,3,NaN,259.100,NaN,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN,10.446,9.212 kr.
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,10.075 / 8.884 kr.,NaN,1972,1.350.000,D,2,NaN,164.300,NaN,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN,10.075,8.884 kr.
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1132]:
df_clean.drop(axis = 1, columns = owner_fee_col, inplace = True)

Finally let's look  at the Kælder (basement) columns before renaming everything.

In [1133]:
basement_cols = ['Kælderareal', 'Kælderrum']
df_clean[basement_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8515 entries, 0 to 51069
Data columns (total 2 columns):
Kælderareal    298 non-null object
Kælderrum      5 non-null object
dtypes: object(2)
memory usage: 199.6+ KB


In [1134]:
print_unique_values(df_clean, basement_cols)

Kælderareal: [nan '27  m2' '61  m2' '97  m2' '102  m2' '40  m2' '84  m2' '112  m2'
 '88  m2' '163  m2' '120  m2' '55  m2' '28  m2' '85  m2' '140  m2'
 '82  m2' '46  m2' '107  m2' '52  m2' '75  m2' '64  m2' '80  m2' '58  m2'
 '135  m2' '34  m2' '126  m2' '105  m2' '86  m2' '95  m2' '87  m2'
 '132  m2' '106  m2' '66  m2' '14  m2' '118  m2' '109  m2' '104  m2'
 '6  m2' '26  m2' '33  m2' '67  m2' '108  m2' '8  m2' '5  m2' '13  m2'
 '29  m2' '76  m2' '41  m2' '156  m2' '90  m2' '74  m2' '3  m2' '71  m2'
 '60  m2' '10  m2' '12  m2' '2  m2' '50  m2' '15  m2' '18  m2' '42  m2'
 '65  m2' '56  m2' '100  m2' '16  m2' '115  m2' '68  m2' '17  m2' '93  m2'
 '110  m2' '122  m2' '70  m2' '9  m2' '73  m2' '92  m2' '72  m2' '20  m2'
 '94  m2' '4  m2' '151  m2' '54  m2' '36  m2' '30  m2' '179  m2' '25  m2'
 '79  m2' '45  m2' '53  m2' '111  m2' '35  m2' '21  m2' '7  m2' '130  m2'
 '11  m2' '44  m2' '32  m2' '98  m2' '24  m2' '145  m2' '39  m2' '78  m2'
 '133  m2' '101  m2' '89  m2' '96  m2' '48  m2' '22  

Clearly not the same feature - let's just drop the Kælderrum and rename the other column.

In [1135]:
df_clean.drop(axis = 1, columns = ['Kælderrum'], inplace = True)
df_clean.rename(columns = {"Kælderareal": "Basement"}, inplace = True)
df_clean.head()

,adresse,boligOrGrundAreal,city,ejendomstypePrimaerNicename,isNew,lat,lng,postal,price,sagsnummer,Aconto forbrug pr. måned,DistanceToShopping,Afstand til off. transport,DistanceToSchool,Afstand til skov,Afstand til vand,Balcony,Antal plan,Antal rum,Antal toiletter,Connection,Boligareal,Byggeår,Ejendomsværdi i kr.,Energimærke,Etage,Grundareal,Heraf grundværdi i kr.,Sewers,Basement,Prisudvikling,Tilbehør,Heating,Windows,Vurderingsår,Exterior,NetOwnerFee,GrossOwnerFee
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,1991,780.000,B,2,NaN,168.300,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN,5.249,4.628 kr.
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,1885,1.100.000,D,3,NaN,145.000,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN,8.313,7.331 kr.
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,1988,1.600.000,C,3,NaN,259.100,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN,10.446,9.212 kr.
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,1972,1.350.000,D,2,NaN,164.300,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN,10.075,8.884 kr.
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


And finally let's rename the remaining columns:

In [1137]:
df_clean.rename(columns = {"adresse": "Address", "boligOrGrundAreal": "Area", 
                           "city": "City", "ejendomstypePrimaerNicename": "HomeType", 
                           "lat": "Latitude", "lng": "Longitude", "postal": "PostalCode", 
                           "price": "Price", "sagsnummer": "ID",
                           "Aconto forbrug  pr. måned": "PaymentOnAccount", 
                           "Afstand til off. transport": "DistanceToTransportation", 
                           "Afstand til skov": "DistanceToForest", "Etage": "Level",
                           "Afstand til vand": "DistanceToWater", "Antal plan": "Floors",
                           "Antal rum": "Rooms", "Antal toiletter": "Toilets", "Boligareal": "LivingArea",
                           "Byggeår": "YearOfConstruction", "Ejendomsværdi i kr.": "EstateValue",
                           "Energimærke": "EnergyRating", "Grundareal": "GroundArea",
                           "Heraf grundværdi i kr.": "LandValue", "Prisudvikling": "PriceDevelopment", 
                           "Tilbehør": "Accessories", "Vurderingsår": "YearOfValuation"},
               inplace = True)
df_clean.head()

,Address,Area,City,HomeType,isNew,Latitude,Longitude,PostalCode,Price,ID,PaymentOnAccount,DistanceToShopping,DistanceToTransportation,DistanceToSchool,DistanceToForest,DistanceToWater,Balcony,Floors,Rooms,Toilets,Connection,LivingArea,YearOfConstruction,EstateValue,EnergyRating,Level,GroundArea,LandValue,Sewers,Basement,PriceDevelopment,Accessories,Heating,Windows,YearOfValuation,Exterior,NetOwnerFee,GrossOwnerFee
0,"A.D. Jørgensens Vej 75, 2. 1.",35.0,Frederiksberg,Ejerlejlighed,False,55.680726,12.494705,2000,1.350.000,1300000111,NaN,1.100 m,150 m,2.600 m,NaN,NaN,No,1,1,1,NaN,35 m2,1991,780.000,B,2,NaN,168.300,NaN,NaN,-7%,Alle nuværende hårde hvidevarer i lejligheden ...,NaN,NaN,2018,NaN,5.249,4.628 kr.
1,"Holger Danskes Vej 14, 3. th.",46.0,Frederiksberg,Ejerlejlighed,False,55.686615,12.538356,2000,2.145.000,1740000062,NaN,150 m,150 m,100 m,NaN,NaN,NaN,1,2,1,NaN,46 m2,1885,1.100.000,D,3,NaN,145.000,NaN,NaN,-6%,Intuition gaskomfur komfur - Matsui Fridge Do...,NaN,NaN,2018,NaN,8.313,7.331 kr.
11,"Lyøvej 5, st.. tv.",60.0,Frederiksberg,Ejerlejlighed,False,55.682940,12.524527,2000,2.875.000,1300000128,NaN,650 m,100 m,550 m,NaN,NaN,No,1,2,1,NaN,77 m2,1988,1.600.000,C,3,NaN,259.100,NaN,NaN,-4%,emhætte - bordkomfur glaskeramisk - indbygni...,NaN,NaN,2018,NaN,10.446,9.212 kr.
12,"H. Schneekloths Vej 13, 5. th.",56.0,Frederiksberg,Ejerlejlighed,False,55.679928,12.506927,2000,2.750.000,130D01015,NaN,50 m,150 m,550 m,NaN,NaN,Yes,1,2,1,NaN,74 m2,1972,1.350.000,D,2,NaN,164.300,NaN,NaN,0%,De i lejligheden hårde hvidevarer medfølger i ...,NaN,NaN,2018,NaN,10.075,8.884 kr.
30,"Howitzvej 61, 3. th.",67.0,Frederiksberg,Ejerlejlighed,True,55.680209,12.523998,2000,3.195.000,1300000176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1138]:
df_clean.to_csv('home_data_clean_final.csv')